In [ ]:
import pandas as pd
import numpy as np
#import sklearn
import datetime
import time
import matplotlib.pyplot as plt
#from statistics import mean
from scipy.optimize import curve_fit
import pytz
from sklearn.metrics import r2_score, mean_absolute_error, median_absolute_error, max_error
from scipy.stats import linregress
import csv
import math

In [ ]:
#create a table of grades
Grades=np.zeros((10,3))
Students=['S01','S02','S03','S04','S05','S06','S07','S08','S09','S10']
with open('StudentGrades.txt') as f:
    lines = f.readlines()
    ist=0   # student index
    iex=0   # exam index
    for line in lines:
      # print(line)
      for st in Students: 
        if (st in line): # is text like S01 in line
          #print(line)
          i=int(line[-4:-1])  # last 3 chars 
          #print(ist,iex,st,i)
          Grades[ist,iex]=i
          ist=ist+1
          if(ist >9):
            ist=0
            iex=iex+1
            
print(Grades)
Grade2=Grades
for i in range(10): 
    Grade2[i,2]=Grades[i,2]/2   # scale the final grades
#print(Grade2)

In [ ]:
def read_data(filenl):

# filenl list of filenames, 
# return X data, X_time starting times, X_fr sampling Freq, X_mean average, X_std standard dev, X_var variance
  X_time=[]
  X_fr=[]
  X=[]
  X_mean=[]
  X_std=[]
  X_var=[]
  for i in filenl:
    D0 = np.loadtxt(i)
    X_time.append(D0[0])  # first  start time 
    X_fr.append(D0[1])    # sampling freq 
    data=D0[2:]           # data
    X.append(data)
    X_mean.append(np.mean(data))
    X_std.append(np.std(data))
    X_var.append(np.var(data))
    print(i,' mean ',np.mean(data),' std ',np.std(data),'var',np.var(data),' data len',len(data))

  for i in range(len(X_time)): 
    print('start time ',datetime.datetime.fromtimestamp(X_time[i],pytz.timezone("US/Central")))
  print('Frequency ',X_fr, 'in Hz')
 
  return X,X_time,X_fr,X_mean,X_std,X_var

In [ ]:
def read_data_only(filenl):

# filenl list of filenames, 
# return X data, X_time starting times, X_fr sampling Freq

  X_time=[]
  X_fr=[]
  X=[]
  for i in filenl:
    D0 = np.loadtxt(i)
    X_time.append(D0[0])
    X_fr.append(D0[1])
    data=D0[2:]
    X.append(data)
    print(i,' mean ',np.mean(data),' std ',np.std(data),'var',np.var(data),' data len',len(data))

  for i in range(len(X_time)): 
    print('start time ',datetime.datetime.fromtimestamp(X_time[i],pytz.timezone("US/Central")))
  print('Frequency ',X_fr, 'in Hz')
 
  return X,X_time,X_fr

In [ ]:
def read_data_IBI(HR):
# read time, and data 
# HR file names, 
# return X data, X_stime starting times, X_time sampling times, X_mean average, X_std standard dev 

  X_time=[]
  X_stime=[]
  X=[]
  X_mean=[]
  X_std=[]
  for i in HR:
    with open(i) as file:
      line = file.readline()
      fields = line.split(',')
      X_stime.append(float(fields[0]))   # from first line get the starting time
    D0 = np.loadtxt(i,delimiter=',',skiprows=1)   # skip the first line
    X_time.append(D0[:,0])   # first collum time
#    X_fr.append(D0[1])  # no freq
    data=D0[:,1]
    X.append(data)    # second collumn  data 
    md=np.mean(data)
    sd=np.std(data)
    X_mean.append(md)
    X_std.append(sd)
    print(i,' mean ',md,' std ',sd,' data len',len(data),'last time',D0[-1,0])

  for i in range(len(X_stime)): 
    print('start time ',datetime.datetime.fromtimestamp(X_stime[i],pytz.timezone("US/Central")))

 
  return X,X_time,X_stime,X_mean,X_std

In [ ]:
def smooth_data(X_in,X_fr,atime):
    
#  X_in input data, atime aver time in sec, X_fr sampling Frew,  
# return  X_smoo smooth data, X_time time steps,

  X_smoo=[]  # smooth HR
  X_time=[]
  for k in range(10):
     window=int(atime*X_fr[k])   # 2 min aver, points 120s*sampling
     midw=int(window/2)
     aver0=[]
     xa=[]
     for i in range(0,len(X_in[k]),window):  # 2 min steps, sampling X_fr Hz
       xa.append((i+midw)/X_fr[k]/3600)
       aver0.append(np.mean(X_in[k][i:i+window]))  # 2 min average
      
     X_smoo.append(aver0)
     X_time.append(xa)
    
  return X_smoo,X_time

In [ ]:
def aver_data(X_in,X_fr,time0,time1):
    
#  X_in input data, time0 start time in sec, time1 end time, X_fr sampling Freq,  
# return  X_aver average, X_std standard dev and X_var variance

  X_aver=[]  # average of X_in
  X_std=[]
  X_var=[]
  for k in range(len(X_fr)):
     i0=int(time0*X_fr[k]) 
     i1=int(time1*X_fr[k]) 
     X_aver.append(np.mean(X_in[k][i0:i1]))  # time0 to time1 average
     X_std.append(np.std(X_in[k][i0:i1]))
     X_var.append(np.var(X_in[k][i0:i1]))
        
  return X_aver,X_std,X_var

In [ ]:
def aver_data_IBI(X_in,X_time,time0,time1):
    
# averaging for IBI 
#  X_in input data, time0 start time in sec, time1 end time, X_time sampling time, NOT regular 
# return  X_aver average and X_std, standard dev

  X_aver=[]  # average of X_in
  X_std=[]
  for k in range(10):
#    print(k,X_time[k][0])
    aver=0.0
    for i in range(len(X_in[k])): 
      if((X_time[k][i] > time0)and(X_time[k][i] < time1)): 
        if i == len(X_in[k])-1:
            dt = 0
        else:
            dt=X_time[k][i+1]-X_time[k][i] # use the time intervall to weight the values
            aver=aver+dt*X_in[k][i]
    aver=aver/(time1-time0)
    X_aver.append(aver)  # 2 min average
    sqd=0.0
    for i in range(len(X_in[k])):
        if((X_time[k][i] > time0)and(X_time[k][i] < time1)): 
            if i == len(X_in[k])-1:
                dt=0
            else:
                dt=X_time[k][i+1]-X_time[k][i] # use the time intervall to weight the values
                sqd=sqd+dt*(X_in[k][i]-aver)**2 #sqd square data
                #print(dt,i)
    sqd = sqd/(time1-time0)
    #print(sqd)
    #print(dt)
    X_std.append(math.sqrt(sqd))
  return X_aver, X_std

In [ ]:
#code found: https://www.geeksforgeeks.org/how-to-convert-datetime-to-unix-timestamp-in-python/

exam_time_M1 = datetime.datetime(2018,10,13,17,0) #start klo 09:00 CT -> used klo 17:00 for correct unix time
print('exam time =>', exam_time_M1)
unix_M1 = time.mktime(exam_time_M1.timetuple())
print('unix timestamp =>',unix_M1)

exam_time_M2 = datetime.datetime(2018,11,10,17,0) #start klo 09:00 CT -> used klo 17:00 for correct unix time
print('exam time =>', exam_time_M2)
unix_M2 = time.mktime(exam_time_M2.timetuple())
print('unix timestamp =>',unix_M2)

exam_time_F = datetime.datetime(2018,12,5,18,45) #start klo 10:45 CT -> used klo 18:45 for correct unix time
print('exam time =>', exam_time_F)
unix_F = time.mktime(exam_time_F.timetuple())
print('unix timestamp =>',unix_F)

In [ ]:
#test, that unix time is right in US CT timezone
print(unix_M1, unix_M2, unix_F)
print('start time M1_HR',datetime.datetime.fromtimestamp(unix_M1,pytz.timezone("US/Central")))
print('start time M2_HR',datetime.datetime.fromtimestamp(unix_M2,pytz.timezone("US/Central")))
print('start time F_HR',datetime.datetime.fromtimestamp(unix_F,pytz.timezone("US/Central")))

Start reading the data using read_data/read_data_only or read_data_IBI

In [ ]:
EX='Midterm 1/'  # Exam 
PARAM='HR' # parameter
HR=["Data/S1/"+EX+PARAM+".csv","Data/S2/"+EX+PARAM+".csv","Data/S3/"+EX+PARAM+".csv",
    "Data/S4/"+EX+PARAM+".csv","Data/S5/"+EX+PARAM+".csv","Data/S6/"+EX+PARAM+".csv",
     "Data/S7/"+EX+PARAM+".csv","Data/S8/"+EX+PARAM+".csv","Data/S9/"+EX+PARAM+".csv",
     "Data/S10/"+EX+PARAM+".csv"]

M1_HR,M1_HR_time,M1_HR_fr,M1_HR_mean,M1_HR_std,M1_HR_var = read_data(HR)  # return data, time, fr, mean, std

for i in range(10):
    t0 = 0
    t1 = unix_M1-M1_HR_time[i]
    M1_HR_mean_bf, M1_HR_std_bf, M1_HR_var_bf = aver_data(M1_HR,M1_HR_fr,t0,t1) # HR before M1 exam
    t2 = t1 + 5400
    M1_HR_mean_ex, M1_HR_std_ex, M1_HR_var_ex = aver_data(M1_HR,M1_HR_fr,t1,t2) # HR during M1 exam
    t3 = int(len(M1_HR[i]))
    M1_HR_mean_af, M1_HR_std_af, M1_HR_var_af = aver_data(M1_HR,M1_HR_fr,t2,t3) # HR after M1 exam
print('Heart rate BEFORE ',EX,' exam \n', M1_HR_mean_bf, np.mean(M1_HR_mean_bf))
print('Heart rate std BEFORE ',EX,' exam \n', M1_HR_std_bf, np.mean(M1_HR_std_bf))
print('Heart rate var BEFORE ',EX,' exam \n', M1_HR_var_bf, np.mean(M1_HR_var_bf))
print()
print('Heart rate DURING ',EX,' exam \n', M1_HR_mean_ex, np.mean(M1_HR_mean_ex))
print('Heart rate std DURING ',EX,' exam \n', M1_HR_std_ex, np.mean(M1_HR_std_ex))
print('Heart rate var DURING ',EX,' exam \n', M1_HR_var_ex, np.mean(M1_HR_var_ex))
print()
print('Heart rate AFTER ',EX,' exam \n', M1_HR_mean_af, np.mean(M1_HR_mean_af))
print('Heart rate std AFTER ',EX,' exam \n', M1_HR_std_af, np.mean(M1_HR_std_af))
print('Heart rate var AFTER ',EX,' exam \n', M1_HR_var_af, np.mean(M1_HR_var_af))

In [ ]:
EX='Midterm 2/'  # Exam 
PARAM='HR' # parameter
HR=["Data/S1/"+EX+PARAM+".csv","Data/S2/"+EX+PARAM+".csv","Data/S3/"+EX+PARAM+".csv",
    "Data/S4/"+EX+PARAM+".csv","Data/S5/"+EX+PARAM+".csv","Data/S6/"+EX+PARAM+".csv",
     "Data/S7/"+EX+PARAM+".csv","Data/S8/"+EX+PARAM+".csv","Data/S9/"+EX+PARAM+".csv",
     "Data/S10/"+EX+PARAM+".csv"]

M2_HR,M2_HR_time,M2_HR_fr,M2_HR_mean,M2_HR_std,M2_HR_var = read_data(HR)  # return data, time, fr, mean, std

for i in range(10):
    t0 = 0
    t1 = unix_M2-M2_HR_time[i]
    M2_HR_mean_bf, M2_HR_std_bf, M2_HR_var_bf = aver_data(M2_HR,M2_HR_fr,t0,t1) # HR before M2 exam
    t2 = t1 + 5400
    M2_HR_mean_ex, M2_HR_std_ex, M2_HR_var_ex = aver_data(M2_HR,M2_HR_fr,t1,t2) # HR during M2 exam
    t3 = int(len(M2_HR[i]))
    M2_HR_mean_af, M2_HR_std_af, M2_HR_var_af = aver_data(M2_HR,M2_HR_fr,t2,t3) # HR during M2 exam
print('Heart rate BEFORE ',EX,' exam \n', M2_HR_mean_bf, np.mean(M2_HR_mean_bf))
print('Heart rate std BEFORE ',EX,' exam \n', M2_HR_std_bf, np.mean(M2_HR_std_bf))
print('Heart rate var BEFORE ',EX,' exam \n', M2_HR_var_bf, np.mean(M2_HR_var_bf))
print()
print('Heart rate DURING ',EX,' exam \n', M2_HR_mean_ex, np.mean(M2_HR_mean_ex))
print('Heart rate std DURING ',EX,' exam \n', M2_HR_std_ex, np.mean(M2_HR_std_ex))
print('Heart rate var DURING ',EX,' exam \n', M2_HR_var_ex, np.mean(M2_HR_var_ex))
print()
print('Heart rate AFTER ',EX,' exam \n', M2_HR_mean_af, np.mean(M2_HR_mean_af))
print('Heart rate std AFTER ',EX,' exam \n', M2_HR_std_af, np.mean(M2_HR_std_af))
print('Heart rate var AFTER ',EX,' exam \n', M2_HR_var_af, np.mean(M2_HR_var_af))

In [ ]:
EX='Final/'  # Exam 
PARAM='HR' # parameter
HR=["Data/S1/"+EX+PARAM+".csv","Data/S2/"+EX+PARAM+".csv","Data/S3/"+EX+PARAM+".csv",
    "Data/S4/"+EX+PARAM+".csv","Data/S5/"+EX+PARAM+".csv","Data/S6/"+EX+PARAM+".csv",
     "Data/S7/"+EX+PARAM+".csv","Data/S8/"+EX+PARAM+".csv","Data/S9/"+EX+PARAM+".csv",
     "Data/S10/"+EX+PARAM+".csv"]

F_HR,F_HR_time,F_HR_fr,F_HR_mean,F_HR_std,F_HR_var = read_data(HR)  # return data, time, fr, mean, std
#print(F_HR_fr,F_HR_mean)

for i in range(10):
    t0 = 0
    t1 = unix_F-F_HR_time[i]
    F_HR_mean_bf, F_HR_std_bf,F_HR_var_bf = aver_data(F_HR,F_HR_fr,t0,t1) # HR before F exam
    t2 = t1 + 5400*2
    F_HR_mean_ex, F_HR_std_ex,F_HR_var_ex = aver_data(F_HR,F_HR_fr,t1,t2) # HR during F exam
    t3 = int(len(F_HR[i]))
    F_HR_mean_af, F_HR_std_af,F_HR_var_af = aver_data(F_HR,F_HR_fr,t2,t3) # HR during F exam
print('Heart rate BEFORE ',EX,' exam \n', F_HR_mean_bf, np.mean(F_HR_mean_bf))
print('Heart rate std BEFORE ',EX,' exam \n', F_HR_std_bf, np.mean(F_HR_std_bf))
print('Heart rate var BEFORE ',EX,' exam \n', F_HR_var_bf, np.mean(F_HR_var_bf))
print()
print('Heart rate DURING ',EX,' exam \n', F_HR_mean_ex, np.mean(F_HR_mean_ex))
print('Heart rate std DURING ',EX,' exam \n', F_HR_std_ex, np.mean(F_HR_std_ex))
print('Heart rate var DURING ',EX,' exam \n', F_HR_var_ex, np.mean(F_HR_var_ex))
print()
print('Heart rate AFTER ',EX,' exam \n', F_HR_mean_af, np.mean(F_HR_mean_af))
print('Heart rate std AFTER ',EX,' exam \n', F_HR_std_af, np.mean(F_HR_std_af))
print('Heart rate var AFTER ',EX,' exam \n', F_HR_var_af, np.mean(F_HR_var_af))

In [ ]:
EX='Midterm 1/'  # Exam 
PARAM='IBI' # parameter
HR=["Data/S1/"+EX+PARAM+".csv","Data/S2/"+EX+PARAM+".csv","Data/S3/"+EX+PARAM+".csv",
    "Data/S4/"+EX+PARAM+".csv","Data/S5/"+EX+PARAM+".csv","Data/S6/"+EX+PARAM+".csv",
     "Data/S7/"+EX+PARAM+".csv","Data/S8/"+EX+PARAM+".csv","Data/S9/"+EX+PARAM+".csv",
     "Data/S10/"+EX+PARAM+".csv"]

M1_IB,M1_IB_stime,M1_IB_time,M1_IB_mean,M1_IB_std = read_data_IBI(HR)  # return data, time, start time, mean, std
#print(F_IB_stime[-1][:])

for i in range(10):
    t0 = 0
    t1 = unix_M1-M1_IB_time[i]
    M1_IB_mean_bf, M1_IB_std_bf = aver_data_IBI(M1_IB,M1_IB_stime,t0,t1) # IBI at M1 exam
    t2 = t1 + 5400
    M1_IB_mean_ex, M1_IB_std_ex = aver_data_IBI(M1_IB,M1_IB_stime,t1,t2) # IBI during M1 exam
    #M1_IB_std_ex = std_data_IBI(M1_IB_mean_ex)
    t3 = M1_IB_stime[i][-1]
    M1_IB_mean_af, M1_IB_std_af = aver_data_IBI(M1_IB,M1_IB_stime,t2,t3) # IBI during M1 exam
    #M1_IB_std_af = std_data_IBI(M1_IB_mean_af)
print('IBI BEFORE ',EX,' exam \n', M1_IB_mean_bf, np.mean(M1_IB_mean_bf))
print('IBI std BEFORE ',EX,' exam \n', M1_IB_std_bf, np.mean(M1_IB_std_bf))
print()
print('IBI DURING ',EX,' exam \n', M1_IB_mean_ex, np.mean(M1_IB_mean_ex))
print('IBI std DURING ',EX,' exam \n', M1_IB_std_ex, np.mean(M1_IB_std_ex))
print()
print('IBI AFTER ',EX,' exam \n', M1_IB_mean_af, np.mean(M1_IB_mean_af))
print('IBI std AFTER ',EX,' exam \n', M1_IB_std_af, np.mean(M1_IB_std_af))

In [ ]:
EX='Midterm 2/'  # Exam 
PARAM='IBI' # parameter
HR=["Data/S1/"+EX+PARAM+".csv","Data/S2/"+EX+PARAM+".csv","Data/S3/"+EX+PARAM+".csv",
    "Data/S4/"+EX+PARAM+".csv","Data/S5/"+EX+PARAM+".csv","Data/S6/"+EX+PARAM+".csv",
     "Data/S7/"+EX+PARAM+".csv","Data/S8/"+EX+PARAM+".csv","Data/S9/"+EX+PARAM+".csv",
     "Data/S10/"+EX+PARAM+".csv"]

M2_IB,M2_IB_stime,M2_IB_time,M2_IB_mean,M2_IB_std = read_data_IBI(HR)  # return data, time, samp time, mean, std

for i in range(10):
    t0 = 0
    t1 = unix_M2-M2_IB_time[i]
    M2_IB_mean_bf, M2_IB_std_bf = aver_data_IBI(M2_IB,M2_IB_stime,t0,t1) # IBI at M2 exam
    t2 = t1 + 5400
    M2_IB_mean_ex, M2_IB_std_ex = aver_data_IBI(M2_IB,M2_IB_stime,t1,t2) # IBI during M2 exam
    t3 = M2_IB_stime[i][-1]
    M2_IB_mean_af, M2_IB_std_af = aver_data_IBI(M2_IB,M2_IB_stime,t2,t3) # IBI during M2 exam
print('IBI BEFORE ',EX,' exam \n', M2_IB_mean_bf, np.mean(M2_IB_mean_bf))
print('IBI std BEFORE ',EX,' exam \n', M2_IB_std_bf, np.mean(M2_IB_std_bf))
print()
print('IBI DURING ',EX,' exam \n', M2_IB_mean_ex, np.mean(M2_IB_mean_ex))
print('IBI std DURING ',EX,' exam \n', M2_IB_std_ex, np.mean(M2_IB_std_ex))
print()
print('IBI AFTER ',EX,' exam \n', M2_IB_mean_af, np.mean(M2_IB_mean_af))
print('IBI std AFTER ',EX,' exam \n', M2_IB_std_af, np.mean(M2_IB_std_af))

In [ ]:
EX='Final/'  # Exam 
PARAM='IBI' # parameter
HR=["Data/S1/"+EX+PARAM+".csv","Data/S2/"+EX+PARAM+".csv","Data/S3/"+EX+PARAM+".csv",
    "Data/S4/"+EX+PARAM+".csv","Data/S5/"+EX+PARAM+".csv","Data/S6/"+EX+PARAM+".csv",
     "Data/S7/"+EX+PARAM+".csv","Data/S8/"+EX+PARAM+".csv","Data/S9/"+EX+PARAM+".csv",
     "Data/S10/"+EX+PARAM+".csv"]

F_IB,F_IB_stime,F_IB_time,F_IB_mean,F_IB_std = read_data_IBI(HR)  # return data, start time, samp time, mean, std

for i in range(10):
    t0 = 0
    t1 = unix_F-F_IB_time[i]
    F_IB_mean_bf, F_IB_std_bf = aver_data_IBI(F_IB,F_IB_stime,t0,t1) # IBI at M2 exam
    t2 = t1 + 5400*2
    F_IB_mean_ex, F_IB_std_ex = aver_data_IBI(F_IB,F_IB_stime,t1,t2) # IBI during M2 exam
    t3 = F_IB_stime[i][-1]
    F_IB_mean_af, F_IB_std_af = aver_data_IBI(F_IB,F_IB_stime,t2,t3) # IBI during M2 exam
print('IBI BEFORE ',EX,' exam \n', F_IB_mean_bf, np.mean(F_IB_mean_bf))
print('IBI std BEFORE ',EX,' exam \n', F_IB_std_bf, np.mean(F_IB_std_bf))
print()
print('IBI DURING ',EX,' exam \n', F_IB_mean_ex, np.mean(F_IB_mean_ex))
print('IBI std DURING ',EX,' exam \n', F_IB_std_ex, np.mean(F_IB_std_ex))
print()
print('IBI AFTER ',EX,' exam \n', F_IB_mean_af, np.mean(F_IB_mean_af))
print('IBI std AFTER ',EX,' exam \n', F_IB_std_af, np.mean(F_IB_std_af))

In [ ]:
EX='Midterm 1/'  # Exam 
PARAM='TEMP' # parameter
TE=["Data/S1/"+EX+PARAM+".csv","Data/S2/"+EX+PARAM+".csv","Data/S3/"+EX+PARAM+".csv",
    "Data/S4/"+EX+PARAM+".csv","Data/S5/"+EX+PARAM+".csv","Data/S6/"+EX+PARAM+".csv",
     "Data/S7/"+EX+PARAM+".csv","Data/S8/"+EX+PARAM+".csv","Data/S9/"+EX+PARAM+".csv",
     "Data/S10/"+EX+PARAM+".csv"]

M1_TE,M1_TE_time,M1_TE_fr = read_data_only(TE) # return data, time, fr
#M1_TE,M1_TE_time,M1_TE_fr,M1_TE_mean,M1_TE_std,M1_TE_var = read_data(TE)

for i in range(10):
    t0 = 0
    t1 = unix_M1-M1_TE_time[i]
    M1_TE_mean_bf, M1_TE_std_bf,M1_TE_var_bf = aver_data(M1_TE,M1_TE_fr,t0,t1) # TEMP before M1 exam
    t2 = t1 + 5400
    M1_TE_mean_ex, M1_TE_std_ex,M1_TE_var_ex = aver_data(M1_TE,M1_TE_fr,t1,t2) # TEMP during M1 exam
    t3 = int(len(M1_TE[i]))/4
    M1_TE_mean_af, M1_TE_std_af,M1_TE_var_af = aver_data(M1_TE,M1_TE_fr,t2,t3) # TEMP during M1 exam
print('Temp BEFORE ',EX,' exam \n', M1_TE_mean_bf, np.mean(M1_TE_mean_bf))
print('Temp std BEFORE ',EX,' exam \n', M1_TE_std_bf, np.mean(M1_TE_std_bf))
print('Temp var BEFORE ',EX,' exam \n', M1_TE_var_bf, np.mean(M1_TE_var_bf))
print()
print('Temp DURING ',EX,' exam \n', M1_TE_mean_ex, np.mean(M1_TE_mean_ex))
print('Temp std DURING ',EX,' exam \n', M1_TE_std_ex, np.mean(M1_TE_std_ex))
print('Temp var DURING ',EX,' exam \n', M1_TE_var_ex, np.mean(M1_TE_var_ex))
print()
print('Temp AFTER ',EX,' exam \n', M1_TE_mean_af, np.mean(M1_TE_mean_af))
print('Temp std AFTER ',EX,' exam \n', M1_TE_std_af, np.mean(M1_TE_std_af))
print('Temp var AFTER ',EX,' exam \n', M1_TE_var_af, np.mean(M1_TE_var_af))

In [ ]:
EX='Midterm 2/'  # Exam 
PARAM='TEMP' # parameter
TE=["Data/S1/"+EX+PARAM+".csv","Data/S2/"+EX+PARAM+".csv","Data/S3/"+EX+PARAM+".csv",
    "Data/S4/"+EX+PARAM+".csv","Data/S5/"+EX+PARAM+".csv","Data/S6/"+EX+PARAM+".csv",
     "Data/S7/"+EX+PARAM+".csv","Data/S8/"+EX+PARAM+".csv","Data/S9/"+EX+PARAM+".csv",
     "Data/S10/"+EX+PARAM+".csv"]

M2_TE,M2_TE_time,M2_TE_fr,M2_TE_mean,M2_TE_std,M2_TE_var = read_data(TE) # return data, time, fr, mean, std

for i in range(10):
    t0 = 0
    t1 = unix_M2-M2_TE_time[i]
    M2_TE_mean_bf, M2_TE_std_bf, M2_TE_var_bf = aver_data(M2_TE,M2_TE_fr,t0,t1) # HR before M1 exam
    t2 = t1 + 5400
    M2_TE_mean_ex, M2_TE_std_ex, M2_TE_var_ex = aver_data(M2_TE,M2_TE_fr,t1,t2) # HR during M1 exam
    t3 = int(len(M2_TE[i]))/4
    M2_TE_mean_af, M2_TE_std_af, M2_TE_var_af = aver_data(M2_TE,M2_TE_fr,t2,t3) # HR during M1 exam
print('Temp BEFORE ',EX,' exam \n', M2_TE_mean_bf, np.mean(M2_TE_mean_bf))
print('Temp std BEFORE ',EX,' exam \n', M2_TE_std_bf, np.mean(M2_TE_std_bf))
print('Temp var BEFORE ',EX,' exam \n', M2_TE_var_bf, np.mean(M2_TE_var_bf))
print()
print('Temp DURING ',EX,' exam \n', M2_TE_mean_ex, np.mean(M2_TE_mean_ex))
print('Temp std DURING ',EX,' exam \n', M2_TE_std_ex, np.mean(M2_TE_std_ex))
print('Temp var DURING ',EX,' exam \n', M2_TE_var_ex, np.mean(M2_TE_var_ex))
print()
print('Temp AFTER ',EX,' exam \n', M2_TE_mean_af, np.mean(M2_TE_mean_af))
print('Temp std AFTER ',EX,' exam \n', M2_TE_std_af, np.mean(M2_TE_std_af))
print('Temp var AFTER ',EX,' exam \n', M2_TE_var_af, np.mean(M2_TE_var_af))

In [ ]:
EX='Final/'  # Exam 
PARAM='TEMP' # parameter
TE=["Data/S1/"+EX+PARAM+".csv","Data/S2/"+EX+PARAM+".csv","Data/S3/"+EX+PARAM+".csv",
    "Data/S4/"+EX+PARAM+".csv","Data/S5/"+EX+PARAM+".csv","Data/S6/"+EX+PARAM+".csv",
     "Data/S7/"+EX+PARAM+".csv","Data/S8/"+EX+PARAM+".csv","Data/S9/"+EX+PARAM+".csv",
     "Data/S10/"+EX+PARAM+".csv"]

F_TE,F_TE_time,F_TE_fr = read_data_only(TE) # return data, time, fr

for i in range(10):
    t0 = 0
    t1 = unix_F-F_TE_time[i]
    F_TE_mean_bf, F_TE_std_bf, F_TE_var_bf = aver_data(F_TE,F_TE_fr,t0,t1) # HR before M1 exam
    t2 = t1 + 5400*2
    F_TE_mean_ex, F_TE_std_ex, F_TE_var_ex = aver_data(F_TE,F_TE_fr,t1,t2) # HR during M1 exam
    t3 = int(len(F_TE[i]))/4
    F_TE_mean_af, F_TE_std_af, F_TE_var_af = aver_data(F_TE,F_TE_fr,t2,t3) # HR during M1 exam
print('Temp BEFORE ',EX,' exam \n', F_TE_mean_bf, np.mean(F_TE_mean_bf))
print('Temp std BEFORE ',EX,' exam \n', F_TE_std_bf, np.mean(F_TE_std_bf))
print('Temp var BEFORE ',EX,' exam \n', F_TE_var_bf, np.mean(F_TE_var_bf))
print()
print('Temp DURING ',EX,' exam \n', F_TE_mean_ex, np.mean(F_TE_mean_ex))
print('Temp std DURING ',EX,' exam \n', F_TE_std_ex, np.mean(F_TE_std_ex))
print('Temp var DURING ',EX,' exam \n', F_TE_var_ex, np.mean(F_TE_var_ex))
print()
print('Temp AFTER ',EX,' exam \n', F_TE_mean_af, np.mean(F_TE_mean_af))
print('Temp std AFTER ',EX,' exam \n', F_TE_std_af, np.mean(F_TE_std_af))
print('Temp var AFTER ',EX,' exam \n', F_TE_var_af, np.mean(F_TE_var_af))

In [ ]:
EX='Midterm 1/'  # Exam 
PARAM='BVP' # parameter
BVP=["Data/S1/"+EX+PARAM+".csv","Data/S2/"+EX+PARAM+".csv","Data/S3/"+EX+PARAM+".csv",
    "Data/S4/"+EX+PARAM+".csv","Data/S5/"+EX+PARAM+".csv","Data/S6/"+EX+PARAM+".csv",
     "Data/S7/"+EX+PARAM+".csv","Data/S8/"+EX+PARAM+".csv","Data/S9/"+EX+PARAM+".csv",
     "Data/S10/"+EX+PARAM+".csv"]

M1_BVP,M1_BVP_time,M1_BVP_fr,M1_BVP_mean,M1_BVP_std,M1_BVP_var = read_data(BVP)  # return data, time, fr, mean, std

for i in range(10):
    t0 = 0
    t1 = unix_M1-M1_BVP_time[i]
    M1_BVP_mean_bf, M1_BVP_std_bf, M1_BVP_var_bf = aver_data(M1_BVP,M1_BVP_fr,t0,t1) # BVP before M1 exam
    t2 = t1 + 5400
    M1_BVP_mean_ex, M1_BVP_std_ex, M1_BVP_var_ex = aver_data(M1_BVP,M1_BVP_fr,t1,t2) # BVP during M1 exam
    t3 = int(len(M1_BVP[i]))/64
    M1_BVP_mean_af, M1_BVP_std_af, M1_BVP_var_af = aver_data(M1_BVP,M1_BVP_fr,t2,t3) # BVP during M1 exam
print('BVP BEFORE ',EX,' exam \n', M1_BVP_mean_bf, np.mean(M1_BVP_mean_bf))
print('BVP std BEFORE ',EX,' exam \n', M1_BVP_std_bf, np.mean(M1_BVP_std_bf))
print('BVP var BEFORE ',EX,' exam \n', M1_BVP_var_bf, np.mean(M1_BVP_var_bf))
print()
print('BVP DURING ',EX,' exam \n', M1_BVP_mean_ex, np.mean(M1_BVP_mean_ex))
print('BVP std DURING ',EX,' exam \n', M1_BVP_std_ex, np.mean(M1_BVP_std_ex))
print('BVP var DURING ',EX,' exam \n', M1_BVP_var_ex, np.mean(M1_BVP_var_ex))
print()
print('BVP AFTER ',EX,' exam \n', M1_BVP_mean_af, np.mean(M1_BVP_mean_af))
print('BVP std AFTER ',EX,' exam \n', M1_BVP_std_af, np.mean(M1_BVP_std_af))
print('BVP var AFTER ',EX,' exam \n', M1_BVP_var_af, np.mean(M1_BVP_var_af))

In [ ]:
EX='Midterm 2/'  # Exam 
PARAM='BVP' # parameter
BVP=["Data/S1/"+EX+PARAM+".csv","Data/S2/"+EX+PARAM+".csv","Data/S3/"+EX+PARAM+".csv",
    "Data/S4/"+EX+PARAM+".csv","Data/S5/"+EX+PARAM+".csv","Data/S6/"+EX+PARAM+".csv",
     "Data/S7/"+EX+PARAM+".csv","Data/S8/"+EX+PARAM+".csv","Data/S9/"+EX+PARAM+".csv",
     "Data/S10/"+EX+PARAM+".csv"]

M2_BVP,M2_BVP_time,M2_BVP_fr,M2_BVP_mean,M2_BVP_std,M2_BVP_var = read_data(BVP)  # return data, time, fr, mean, std

for i in range(10):
    t0 = 0
    t1 = unix_M2-M2_BVP_time[i]
    M2_BVP_mean_bf, M2_BVP_std_bf, M2_BVP_var_bf = aver_data(M2_BVP,M2_BVP_fr,t0,t1) # BVP before M2 exam
    t2 = t1 + 5400
    M2_BVP_mean_ex, M2_BVP_std_ex, M2_BVP_var_ex = aver_data(M2_BVP,M2_BVP_fr,t1,t2) # BVP during M2 exam
    t3 = int(len(M2_BVP[i]))/64
    M2_BVP_mean_af, M2_BVP_std_af, M2_BVP_var_af = aver_data(M2_BVP,M2_BVP_fr,t2,t3) # BVP during M2 exam
print('BVP BEFORE ',EX,' exam \n', M2_BVP_mean_bf, np.mean(M2_BVP_mean_bf))
print('BVP std BEFORE ',EX,' exam \n', M2_BVP_std_bf, np.mean(M2_BVP_std_bf))
print('BVP var BEFORE ',EX,' exam \n', M2_BVP_var_bf, np.mean(M2_BVP_var_bf))
print()
print('BVP DURING ',EX,' exam \n', M2_BVP_mean_ex, np.mean(M2_BVP_mean_ex))
print('BVP std DURING ',EX,' exam \n', M2_BVP_std_ex, np.mean(M2_BVP_std_ex))
print('BVP var DURING ',EX,' exam \n', M2_BVP_var_ex, np.mean(M2_BVP_var_ex))
print()
print('BVP AFTER ',EX,' exam \n', M2_BVP_mean_af, np.mean(M2_BVP_mean_af))
print('BVP std AFTER ',EX,' exam \n', M2_BVP_std_af, np.mean(M2_BVP_std_af))
print('BVP var AFTER ',EX,' exam \n', M2_BVP_var_af, np.mean(M2_BVP_var_af))

In [ ]:
EX='Final/'  # Exam 
PARAM='BVP' # parameter
BVP=["Data/S1/"+EX+PARAM+".csv","Data/S2/"+EX+PARAM+".csv","Data/S3/"+EX+PARAM+".csv",
    "Data/S4/"+EX+PARAM+".csv","Data/S5/"+EX+PARAM+".csv","Data/S6/"+EX+PARAM+".csv",
     "Data/S7/"+EX+PARAM+".csv","Data/S8/"+EX+PARAM+".csv","Data/S9/"+EX+PARAM+".csv",
     "Data/S10/"+EX+PARAM+".csv"]

F_BVP,F_BVP_time,F_BVP_fr,F_BVP_mean,F_BVP_std,F_BVP_var = read_data(BVP)  # return data, time, fr, mean, std

for i in range(10):
    t0 = 0
    t1 = unix_F-F_BVP_time[i]
    F_BVP_mean_bf, F_BVP_std_bf, F_BVP_var_bf = aver_data(F_BVP,F_BVP_fr,t0,t1) # BVP before F exam
    t2 = t1 + 5400*2
    F_BVP_mean_ex, F_BVP_std_ex, F_BVP_var_ex = aver_data(F_BVP,F_BVP_fr,t1,t2) # BVP during F exam
    t3 = int(len(F_BVP[i]))/64
    F_BVP_mean_af, F_BVP_std_af, F_BVP_var_af = aver_data(F_BVP,F_BVP_fr,t2,t3) # BVP during F exam
print('BVP BEFORE ',EX,' exam \n', F_BVP_mean_bf, np.mean(F_BVP_mean_bf))
print('BVP std BEFORE ',EX,' exam \n', F_BVP_std_bf, np.mean(F_BVP_std_bf))
print('BVP var BEFORE ',EX,' exam \n', F_BVP_var_bf, np.mean(F_BVP_var_bf))
print()
print('BVP DURING ',EX,' exam \n', F_BVP_mean_ex, np.mean(F_BVP_mean_ex))
print('BVP std DURING ',EX,' exam \n', F_BVP_std_ex, np.mean(F_BVP_std_ex))
print('BVP var DURING ',EX,' exam \n', F_BVP_var_ex, np.mean(F_BVP_var_ex))
print()
print('BVP AFTER ',EX,' exam \n', F_BVP_mean_af, np.mean(F_BVP_mean_af))
print('BVP std AFTER ',EX,' exam \n', F_BVP_std_af, np.mean(F_BVP_std_af))
print('BVP var AFTER ',EX,' exam \n', F_BVP_var_af, np.mean(F_BVP_var_af))

In [ ]:
EX='Midterm 1/'  # Exam 
PARAM='EDA' # parameter
EDA=["Data/S1/"+EX+PARAM+".csv","Data/S2/"+EX+PARAM+".csv","Data/S3/"+EX+PARAM+".csv",
    "Data/S4/"+EX+PARAM+".csv","Data/S5/"+EX+PARAM+".csv","Data/S6/"+EX+PARAM+".csv",
     "Data/S7/"+EX+PARAM+".csv","Data/S8/"+EX+PARAM+".csv","Data/S9/"+EX+PARAM+".csv",
     "Data/S10/"+EX+PARAM+".csv"]

M1_EDA,M1_EDA_time,M1_EDA_fr,M1_EDA_mean,M1_EDA_std,M1_EDA_var = read_data(EDA)  # return data, time, fr, mean, std

for i in range(10):
    t0 = 0
    t1 = unix_M1-M1_EDA_time[i]
    M1_EDA_mean_bf, M1_EDA_std_bf, M1_EDA_var_bf = aver_data(M1_EDA,M1_EDA_fr,t0,t1) # EDA before M1 exam
    t2 = t1 + 5400
    M1_EDA_mean_ex, M1_EDA_std_ex, M1_EDA_var_ex = aver_data(M1_EDA,M1_EDA_fr,t1,t2) # EDA during M1 exam
    t3 = int(len(M1_EDA[i]))/4
    M1_EDA_mean_af, M1_EDA_std_af, M1_EDA_var_af = aver_data(M1_EDA,M1_EDA_fr,t2,t3) # EDA during M1 exam
print('EDA BEFORE ',EX,' exam \n', M1_EDA_mean_bf, np.mean(M1_EDA_mean_bf))
print('EDA std BEFORE ',EX,' exam \n', M1_EDA_std_bf, np.mean(M1_EDA_std_bf))
print('EDA var BEFORE ',EX,' exam \n', M1_EDA_var_bf, np.mean(M1_EDA_var_bf))
print()
print('EDA DURING ',EX,' exam \n', M1_EDA_mean_ex, np.mean(M1_EDA_mean_ex))
print('EDA std DURING ',EX,' exam \n', M1_EDA_std_ex, np.mean(M1_EDA_std_ex))
print('EDA var DURING ',EX,' exam \n', M1_EDA_var_ex, np.mean(M1_EDA_var_ex))
print()
print('EDA AFTER ',EX,' exam \n', M1_EDA_mean_af, np.mean(M1_EDA_mean_af))
print('EDA std AFTER ',EX,' exam \n', M1_EDA_std_af, np.mean(M1_EDA_std_af))
print('EDA var AFTER ',EX,' exam \n', M1_EDA_var_af, np.mean(M1_EDA_var_af))

In [ ]:
EX='Midterm 2/'  # Exam 
PARAM='EDA' # parameter
EDA=["Data/S1/"+EX+PARAM+".csv","Data/S2/"+EX+PARAM+".csv","Data/S3/"+EX+PARAM+".csv",
    "Data/S4/"+EX+PARAM+".csv","Data/S5/"+EX+PARAM+".csv","Data/S6/"+EX+PARAM+".csv",
     "Data/S7/"+EX+PARAM+".csv","Data/S8/"+EX+PARAM+".csv","Data/S9/"+EX+PARAM+".csv",
     "Data/S10/"+EX+PARAM+".csv"]

M2_EDA,M2_EDA_time,M2_EDA_fr = read_data_only(EDA)  # return data, time, fr, mean, std

for i in range(10):
    t0 = 0
    t1 = unix_M2-M2_EDA_time[i]
    M2_EDA_mean_bf, M2_EDA_std_bf, M2_EDA_var_bf = aver_data(M2_EDA,M2_EDA_fr,t0,t1) # EDA before M2 exam
    t2 = t1 + 5400
    M2_EDA_mean_ex, M2_EDA_std_ex, M2_EDA_var_ex = aver_data(M2_EDA,M2_EDA_fr,t1,t2) # EDA during M2 exam
    t3 = int(len(M2_EDA[i]))/4
    M2_EDA_mean_af, M2_EDA_std_af, M2_EDA_var_af = aver_data(M2_EDA,M2_EDA_fr,t2,t3) # EDA after M2 exam
print('EDA BEFORE ',EX,' exam \n', M2_EDA_mean_bf, np.mean(M2_EDA_mean_bf))
print('EDA std BEFORE ',EX,' exam \n', M2_EDA_std_bf, np.mean(M2_EDA_std_bf))
print('EDA var BEFORE ',EX,' exam \n', M2_EDA_var_bf, np.mean(M2_EDA_var_bf))
print()
print('EDA DURING ',EX,' exam \n', M2_EDA_mean_ex, np.mean(M2_EDA_mean_ex))
print('EDA std DURING ',EX,' exam \n', M2_EDA_std_ex, np.mean(M2_EDA_std_ex))
print('EDA var DURING ',EX,' exam \n', M2_EDA_var_ex, np.mean(M2_EDA_var_ex))
print()
print('EDA AFTER ',EX,' exam \n', M2_EDA_mean_af, np.mean(M2_EDA_mean_af))
print('EDA std AFTER ',EX,' exam \n', M2_EDA_std_af, np.mean(M2_EDA_std_af))
print('EDA var AFTER ',EX,' exam \n', M2_EDA_var_af, np.mean(M2_EDA_var_af))

In [ ]:
EX='FINAL/'  # Exam 
PARAM='EDA' # parameter
EDA=["Data/S1/"+EX+PARAM+".csv","Data/S2/"+EX+PARAM+".csv","Data/S3/"+EX+PARAM+".csv",
    "Data/S4/"+EX+PARAM+".csv","Data/S5/"+EX+PARAM+".csv","Data/S6/"+EX+PARAM+".csv",
     "Data/S7/"+EX+PARAM+".csv","Data/S8/"+EX+PARAM+".csv","Data/S9/"+EX+PARAM+".csv",
     "Data/S10/"+EX+PARAM+".csv"]

F_EDA,F_EDA_time,F_EDA_fr = read_data_only(EDA)  # return data, time, fr

for i in range(10):
    t0 = 0
    t1 = unix_F-F_EDA_time[i]
    F_EDA_mean_bf, F_EDA_std_bf, F_EDA_var_bf = aver_data(F_EDA,F_EDA_fr,t0,t1) # EDA before F exam
    t2 = t1 + 5400*2
    F_EDA_mean_ex, F_EDA_std_ex, F_EDA_var_ex = aver_data(F_EDA,F_EDA_fr,t1,t2) # EDA during F exam
    t3 = int(len(F_EDA[i]))/4
    F_EDA_mean_af, F_EDA_std_af, F_EDA_var_af = aver_data(F_EDA,F_EDA_fr,t2,t3) # EDA after F exam
print('EDA BEFORE ',EX,' exam \n', F_EDA_mean_bf, np.mean(F_EDA_mean_bf))
print('EDA std BEFORE ',EX,' exam \n', F_EDA_std_bf, np.mean(F_EDA_std_bf))
print('EDA var BEFORE ',EX,' exam \n', F_EDA_var_bf, np.mean(F_EDA_var_bf))
print()
print('EDA DURING ',EX,' exam \n', F_EDA_mean_ex, np.mean(F_EDA_mean_ex))
print('EDA std DURING ',EX,' exam \n', F_EDA_std_ex, np.mean(F_EDA_std_ex))
print('EDA var DURING ',EX,' exam \n', F_EDA_var_ex, np.mean(F_EDA_var_ex))
print()
print('EDA AFTER ',EX,' exam \n', F_EDA_mean_af, np.mean(F_EDA_mean_af))
print('EDA std AFTER ',EX,' exam \n', F_EDA_std_af, np.mean(F_EDA_std_af))
print('EDA var AFTER ',EX,' exam \n', F_EDA_var_af, np.mean(F_EDA_var_af))

In [ ]:
#comparing the mean/std/var values with raw data to the mean/std/var values with smoothed 
M1_HR_smo_bf=[]
M1_HR_smo_ex=[]
M1_HR_smo_af=[]
M1_HR_frs = []
M1_HR_vars_ex = []
for i in range(10):
    M1_HR_frs.append(M1_HR_fr[i]/atime)
for i in range(10):
    t0 = 0
    t1 = unix_M1-M1_HR_time[i]
    M1_HR_smo_bf, M1_HR_stds_bf, M1_HR_vars_bf = aver_data(M1_HR_ave,M1_HR_frs,t0,t1) # HR before M1 exam
    t2 = t1 + 5400
    M1_HR_smo_ex, M1_HR_stds_ex, M1_HR_vars_ex = aver_data(M1_HR_ave,M1_HR_frs,t1,t2) # HR during M1 exam
    t3 = int(len(M1_HR[i]))
    M1_HR_smo_af, M1_HR_stds_af, M1_HR_vars_af = aver_data(M1_HR_ave,M1_HR_frs,t2,t3) # HR after M1 exam
for i in range(10):
    M1_HR_vars_ex.append(M1_HR_stds_ex[i]**2)
print('Heart rate smoothed aver BEFORE ',EX,' exam \n', M1_HR_smo_bf, np.mean(M1_HR_smo_bf))
print('Heart rate smoothed std BEFORE ',EX,' exam \n', M1_HR_stds_bf, np.mean(M1_HR_stds_bf))
print('Heart rate smoothed var BEFORE ',EX,' exam \n', M1_HR_vars_bf, np.mean(M1_HR_vars_bf))
print()
print('Heart rate aver BEFORE ',EX,' exam \n', M1_HR_mean_bf, np.mean(M1_HR_mean_bf))
print('Heart rate std BEFORE ',EX,' exam \n', M1_HR_std_bf, np.mean(M1_HR_std_bf))
print('Heart rate var BEFORE ',EX,' exam \n', M1_HR_var_bf, np.mean(M1_HR_var_bf))
print()
print('Heart rate smoothed aver DURING ',EX,' exam \n', M1_HR_smo_ex, np.mean(M1_HR_smo_ex))
print('Heart rate smoothed std DURING ',EX,' exam \n', M1_HR_stds_ex, np.mean(M1_HR_stds_ex))
print('Heart rate smoothed var DURING ',EX,' exam \n', M1_HR_vars_ex, np.mean(M1_HR_vars_ex))
print()
print('Heart rate aver DURING ',EX,' exam \n', M1_HR_mean_ex, np.mean(M1_HR_mean_ex))
print('Heart rate std DURING ',EX,' exam \n', M1_HR_std_ex, np.mean(M1_HR_std_ex))
print('Heart rate var DURING ',EX,' exam \n', M1_HR_var_ex, np.mean(M1_HR_var_ex))

From here on we start plotting

In [ ]:
atime = 120 # sec 
M1_HR_ave, t_M1_HR = smooth_data(M1_HR,M1_HR_fr,atime)
x1=[]
for i in range(len(M1_HR[0])):
  x1.append(i/M1_HR_fr[0]/3600)

plt.title('M1 HR S1')
plt.plot(x1,M1_HR[0])
plt.xlabel('hour since start')
plt.show()
x1=[]
for i in range(len(M1_TE[0])):
  x1.append(i/M1_TE_fr[0]/3600)

plt.title('M1 TEMP S1')
plt.plot(x1,M1_TE[0])
plt.xlabel('hour since start')
plt.show()

plt.title('M1 IBI S1')
plt.plot(M1_IB_stime[0],M1_IB[0])
plt.xlabel('hours since start')
x_h1 = [0,3600,7200,10800]
labels1 = [0,1,2,3]
plt.xticks(x_h1, labels1)
plt.show()

x1=[]
for i in range(len(M1_EDA[0])):
  x1.append(i/M1_EDA_fr[0]/3600)

plt.plot(x1,M1_EDA[0])
plt.title('M1 EDA S1')
plt.xlabel('hour since start')
plt.show()

x1=[]
for i in range(len(M1_BVP[0])):
  x1.append(i/M1_BVP_fr[0]/3600)

plt.plot(x1,M1_BVP[0])
plt.title('M1 BVP S1')
plt.xlabel('hour since start')
plt.show()

In [ ]:
atime = 120 # sec 
M1_HR_ave, t_M1_HR = smooth_data(M1_HR,M1_HR_fr,atime)
#print(F_HR_ave)
x1=[]
for i in range(len(M1_HR[0])):
  x1.append(i/M1_HR_fr[0]/3600)

x2=[]
for i in range(len(M1_HR[1])):
  x2.append(i/M1_HR_fr[1]/3600)

plt.title(' M1 HR, S1 and S2')
plt.plot(x1,M1_HR[0],x2,M1_HR[1])
plt.xlabel('hour since start')
plt.show()
plt.title(str(atime)+' s average, S1 and S2, M1')
plt.plot(t_M1_HR[0],M1_HR_ave[0],t_M1_HR[1],M1_HR_ave[1])
plt.gca().legend(('student 1','student 2'))
plt.xlabel('hour since start')
plt.show()

In [ ]:
plt.title(str(atime)+' s average, S3 and S4, M1')
plt.plot(t_M1_HR[2],M1_HR_ave[2],t_M1_HR[3],M1_HR_ave[3])
plt.gca().legend(('student 3','student 4'))
plt.xlabel('hour since start')
plt.show()

plt.title(str(atime)+' s average, S5 and S6, M1')
plt.plot(t_M1_HR[4],M1_HR_ave[4],t_M1_HR[5],M1_HR_ave[5])
plt.gca().legend(('student 5','student 6'))
plt.xlabel('hour since start')
plt.show()

plt.title(str(atime)+' s average, S7 and S8, M1')
plt.plot(t_M1_HR[6],M1_HR_ave[6],t_M1_HR[7],M1_HR_ave[7])
plt.gca().legend(('student 7','student 8'))
plt.xlabel('hour since start')
plt.show()

plt.title(str(atime)+' s average, S9 and S10, M1')
plt.plot(t_M1_HR[8],M1_HR_ave[8],t_M1_HR[9],M1_HR_ave[9])
plt.gca().legend(('student 9','student 10'))
plt.xlabel('hour since start')
plt.show()

In [ ]:
#and all plots just for fun
plt.title(str(atime)+' s average, all students, M1')
plt.plot(t_M1_HR[0],M1_HR_ave[0],t_M1_HR[1],M1_HR_ave[1],t_M1_HR[2],M1_HR_ave[2],t_M1_HR[3],M1_HR_ave[3],t_M1_HR[4],M1_HR_ave[4],
        t_M1_HR[5],M1_HR_ave[5],t_M1_HR[6],M1_HR_ave[6],t_M1_HR[7],M1_HR_ave[7],t_M1_HR[8],M1_HR_ave[8],t_M1_HR[9],M1_HR_ave[9])
plt.xlabel('hour since start')
plt.show()

In [ ]:
atime = 120 # sec 
M2_HR_ave, t_M2_HR = smooth_data(M2_HR,M2_HR_fr,atime)
#print(F_HR_ave)
x1=[]
for i in range(len(M2_HR[0])):
  x1.append(i/M2_HR_fr[0]/3600)

x2=[]
for i in range(len(M2_HR[1])):
  x2.append(i/M2_HR_fr[1]/3600)

plt.title(' M2 HR, S1 and S2')
plt.plot(x1,M2_HR[0],x2,M2_HR[1])
plt.xlabel('hour since start')
plt.show()
plt.title(str(atime)+' s average, S1 and S2, M2')
plt.plot(t_M2_HR[0],M2_HR_ave[0],t_M2_HR[1],M2_HR_ave[1])
plt.xlabel('hour since start')
plt.show()

In [ ]:
plt.title(str(atime)+' s average, S3 and S4, M2')
plt.plot(t_M2_HR[2],M2_HR_ave[2],t_M2_HR[3],M2_HR_ave[3])
plt.xlabel('hour since start')
plt.show()

plt.title(str(atime)+' s average, S5 and S6, M2')
plt.plot(t_M2_HR[4],M2_HR_ave[4],t_M2_HR[5],M2_HR_ave[5])
plt.xlabel('hour since start')
plt.show()

plt.title(str(atime)+' s average, S7 and S8, M2')
plt.plot(t_M2_HR[6],M2_HR_ave[6],t_M2_HR[7],M2_HR_ave[7])
plt.gca().legend(('student 7','student 8'))
plt.xlabel('hour since start')
plt.show()

plt.title(str(atime)+' s average, S9 and S10, M2')
plt.plot(t_M2_HR[8],M2_HR_ave[8],t_M2_HR[9],M2_HR_ave[9])
plt.xlabel('hour since start')
plt.show()

In [ ]:
#and all plots just for fun
plt.title(str(atime)+' s average, all students, M2')
plt.plot(t_M2_HR[0],M2_HR_ave[0],t_M2_HR[1],M2_HR_ave[1],t_M2_HR[2],M2_HR_ave[2],t_M2_HR[3],M2_HR_ave[3],t_M2_HR[4],M2_HR_ave[4],
        t_M2_HR[5],M2_HR_ave[5],t_M2_HR[6],M2_HR_ave[6],t_M2_HR[7],M2_HR_ave[7],t_M2_HR[8],M2_HR_ave[8],t_M2_HR[9],M2_HR_ave[9])
plt.xlabel('hour since start')
plt.show()

In [ ]:
#for n in range(len(HR)):
#   plt.title(HR[n])
#   plt.plot(F_HR[n])
#   plt.show()

atime = 120 # sec 
F_HR_ave, t_F_HR = smooth_data(F_HR,F_HR_fr,atime)
#print(F_HR_ave)
x1=[]
for i in range(len(F_HR[0])):
  x1.append(i/F_HR_fr[0]/3600)

x2=[]
for i in range(len(F_HR[1])):
  x2.append(i/F_HR_fr[1]/3600)

plt.title(' Final HR, S1 and S2')
plt.plot(x1,F_HR[0],x2,F_HR[1])
plt.xlabel('hour since start')
plt.show()
plt.title(str(atime)+' s average, S1 and S2')
plt.plot(t_F_HR[0],F_HR_ave[0],t_F_HR[1],F_HR_ave[1])
plt.xlabel('hour since start')
plt.show()

In [ ]:
atime = 120 # sec 
F_HR_ave, t_F_HR = smooth_data(F_HR,F_HR_fr,atime)
#print(F_HR_ave)
x1=[]
for i in range(len(F_HR[0])):
  x1.append(i/F_HR_fr[0]/3600)

x2=[]
for i in range(len(F_HR[1])):
  x2.append(i/F_HR_fr[1]/3600)

plt.title(str(atime)+' s average, S3 and S4, final')
plt.plot(t_F_HR[2],F_HR_ave[2],t_F_HR[3],F_HR_ave[3])
plt.xlabel('hour since start')
plt.show()

plt.title(str(atime)+' s average, S5 and S6, final')
plt.plot(t_F_HR[4],F_HR_ave[4],t_F_HR[5],F_HR_ave[5])
plt.xlabel('hour since start')
plt.show()

plt.title(str(atime)+' s average, S7 and S8, final')
plt.plot(t_F_HR[6],F_HR_ave[6],t_F_HR[7],F_HR_ave[7])
plt.xlabel('hour since start')
plt.show()

plt.title(str(atime)+' s average, S9 and S10, final')
plt.plot(t_F_HR[8],F_HR_ave[8],t_F_HR[9],F_HR_ave[9])
plt.xlabel('hour since start')
plt.show()

In [ ]:
#and all plots just for fun
plt.title(str(atime)+' s average, all students, final')
plt.plot(t_F_HR[0],F_HR_ave[0],t_F_HR[1],F_HR_ave[1],t_F_HR[2],F_HR_ave[2],t_F_HR[3],F_HR_ave[3],t_F_HR[4],F_HR_ave[4],
        t_F_HR[5],F_HR_ave[5],t_F_HR[6],F_HR_ave[6],t_F_HR[7],F_HR_ave[7],t_F_HR[8],F_HR_ave[8],t_F_HR[9],F_HR_ave[9])
plt.xlabel('hour since start')
plt.show()

In [ ]:
atime = 120 # sec 
M1_TE_ave, t_M1_TE = smooth_data(M1_TE,M1_TE_fr,atime)
#print(F_HR_ave)
x1=[]
for i in range(len(M1_TE[0])):
  x1.append(i/M1_TE_fr[0]/3600)

x2=[]
for i in range(len(M1_TE[1])):
  x2.append(i/M1_TE_fr[1]/3600)

plt.title(' M1 TEMP, S1 and S2')
plt.plot(x1,M1_TE[0],x2,M1_TE[1])
plt.gca().legend(('student 1','student 2'))
plt.xlabel('hour since start')
plt.show()
plt.title(str(atime)+' s average, S1 and S2')
plt.plot(t_M1_TE[0],M1_TE_ave[0],'.',t_M1_TE[1],M1_TE_ave[1],'.')
plt.xlabel('hour since start')
plt.show()

In [ ]:
x1=[]
for i in range(len(M1_TE[2])):
  x1.append(i/M1_TE_fr[2]/3600)

x2=[]
for i in range(len(M1_TE[3])):
  x2.append(i/M1_TE_fr[3]/3600)

plt.title(' M1 TEMP, S3 and S4')
plt.plot(x1,M1_TE[2],x2,M1_TE[3])
plt.xlabel('hour since start')
plt.show()

x1=[]
for i in range(len(M1_TE[4])):
  x1.append(i/M1_TE_fr[4]/3600)

x2=[]
for i in range(len(M1_TE[5])):
  x2.append(i/M1_TE_fr[5]/3600)

plt.title(' M1 TEMP, S5 and S6')
plt.plot(x1,M1_TE[4],x2,M1_TE[5])
plt.xlabel('hour since start')
plt.show()

x1=[]
for i in range(len(M1_TE[6])):
  x1.append(i/M1_TE_fr[6]/3600)

x2=[]
for i in range(len(M1_TE[7])):
  x2.append(i/M1_TE_fr[7]/3600)

plt.title(' M1 TEMP, S7 and S8')
plt.plot(x1,M1_TE[6],x2,M1_TE[7])
plt.xlabel('hour since start')
plt.show()

x1=[]
for i in range(len(M1_TE[8])):
  x1.append(i/M1_TE_fr[8]/3600)

x2=[]
for i in range(len(M1_TE[9])):
  x2.append(i/M1_TE_fr[9]/3600)

plt.title(' M1 TEMP, S9 and S10')
plt.plot(x1,M1_TE[8],x2,M1_TE[9])
plt.xlabel('hour since start')
plt.show()

In [ ]:
x1=[]
for i in range(len(M2_TE[0])):
  x1.append(i/M2_TE_fr[0]/3600)

x2=[]
for i in range(len(M2_TE[1])):
  x2.append(i/M2_TE_fr[1]/3600)

plt.title(' M2 TEMP, S1 and S2')
plt.plot(x1,M2_TE[0],x2,M2_TE[1])
plt.gca().legend(('student 1','student 2'))
plt.xlabel('hour since start')
plt.show()

In [ ]:
x1=[]
for i in range(len(M2_TE[2])):
  x1.append(i/M2_TE_fr[2]/3600)

x2=[]
for i in range(len(M2_TE[3])):
  x2.append(i/M2_TE_fr[3]/3600)

plt.title(' M2 TEMP, S3 and S4')
plt.plot(x1,M2_TE[2],x2,M2_TE[3])
plt.xlabel('hour since start')
plt.show()

x1=[]
for i in range(len(M2_TE[4])):
  x1.append(i/M2_TE_fr[4]/3600)

x2=[]
for i in range(len(M2_TE[5])):
  x2.append(i/M2_TE_fr[5]/3600)

plt.title(' M2 TEMP, S5 and S6')
plt.plot(x1,M2_TE[4],x2,M2_TE[5])
plt.xlabel('hour since start')
plt.show()

x1=[]
for i in range(len(M2_TE[6])):
  x1.append(i/M2_TE_fr[6]/3600)

x2=[]
for i in range(len(M2_TE[7])):
  x2.append(i/M2_TE_fr[7]/3600)

plt.title(' M2 TEMP, S7 and S8')
plt.plot(x1,M2_TE[6],x2,M2_TE[7])
plt.xlabel('hour since start')
plt.show()

x1=[]
for i in range(len(M2_TE[8])):
  x1.append(i/M2_TE_fr[8]/3600)

x2=[]
for i in range(len(M2_TE[9])):
  x2.append(i/M2_TE_fr[9]/3600)

plt.title(' M2 TEMP, S9 and S10')
plt.plot(x1,M2_TE[8],x2,M2_TE[9])
plt.xlabel('hour since start')
plt.show()

In [ ]:
atime = 120 # sec 
F_TE_ave, t_F_TE = smooth_data(F_TE,F_TE_fr,atime)
#print(F_HR_ave)
x1=[]
for i in range(len(F_TE[0])):
  x1.append(i/F_TE_fr[0]/3600)

x2=[]
for i in range(len(F_TE[1])):
  x2.append(i/F_TE_fr[1]/3600)

plt.title(' Final TEMP, S1 and S2')
plt.plot(x1,F_TE[0],x2,F_TE[1])
plt.gca().legend(('student 1','student 2'))
plt.xlabel('hour since start')
plt.show()
#plt.title(str(atime)+' s average, S1 and S2')
#plt.plot(t_F_TE[0],F_TE_ave[0],'.',t_F_TE[1],F_TE_ave[1],'.')
#plt.xlabel('hour since start')
#plt.show()

In [ ]:
x1=[]
for i in range(len(F_TE[2])):
  x1.append(i/F_TE_fr[2]/3600)

x2=[]
for i in range(len(F_TE[3])):
  x2.append(i/F_TE_fr[3]/3600)

plt.title(' F TEMP, S3 and S4')
plt.plot(x1,F_TE[2],x2,F_TE[3])
plt.xlabel('hour since start')
plt.show()

x1=[]
for i in range(len(F_TE[4])):
  x1.append(i/F_TE_fr[4]/3600)

x2=[]
for i in range(len(F_TE[5])):
  x2.append(i/F_TE_fr[5]/3600)

plt.title(' F TEMP, S5 and S6')
plt.plot(x1,F_TE[4],x2,F_TE[5])
plt.xlabel('hour since start')
plt.show()

x1=[]
for i in range(len(F_TE[6])):
  x1.append(i/F_TE_fr[6]/3600)

x2=[]
for i in range(len(F_TE[7])):
  x2.append(i/F_TE_fr[7]/3600)

plt.title(' F TEMP, S7 and S8')
plt.plot(x1,F_TE[6],x2,F_TE[7])
plt.xlabel('hour since start')
plt.show()

x1=[]
for i in range(len(F_TE[8])):
  x1.append(i/F_TE_fr[8]/3600)

x2=[]
for i in range(len(F_TE[9])):
  x2.append(i/F_TE_fr[9]/3600)

plt.title(' F TEMP, S9 and S10')
plt.plot(x1,F_TE[8],x2,F_TE[9])
plt.xlabel('hour since start')
plt.show()

In [ ]:
x1=[]
for i in range(len(M1_TE[0])):
  x1.append(i/M1_TE_fr[0]/3600)

x2=[]
for i in range(len(M1_TE[1])):
  x2.append(i/M1_TE_fr[1]/3600)

plt.title(' M1 TEMP, S1 and S2')
plt.plot(x1,M1_TE[0],x2,M1_TE[1])
plt.gca().legend(('student 1','student 2'))
plt.xlabel('hour since start')
plt.show()

x1=[]
for i in range(len(M2_TE[0])):
  x1.append(i/M2_TE_fr[0]/3600)

x2=[]
for i in range(len(M2_TE[1])):
  x2.append(i/M2_TE_fr[1]/3600)
 
plt.title(' M2 TEMP, S1 and S2')
plt.plot(x1,M2_TE[0],x2,M2_TE[1])
plt.gca().legend(('student 1','student 2'))
plt.xlabel('hour since start')
plt.show()

x1=[]
for i in range(len(F_TE[0])):
  x1.append(i/F_TE_fr[0]/3600)

x2=[]
for i in range(len(F_TE[1])):
  x2.append(i/F_TE_fr[1]/3600)

plt.title(' Final TEMP, S1 and S2')
plt.plot(x1,F_TE[0],x2,F_TE[1])
plt.gca().legend(('student 1','student 2'))
plt.axvline(x = 0.5, color = 'b')
plt.xlabel('hour since start')
plt.show()


In [ ]:
plt.title(' M1 IBI, S1 and S2')
plt.plot(M1_IB_stime[0],M1_IB[0],M1_IB_stime[1],M1_IB[1])
plt.xlabel('hours since start')
x_h1 = [0,3600,7200,10800]
labels1 = [0,1,2,3]
plt.xticks(x_h1, labels1)
plt.axvline(x = 3834.0, color = 'm')
plt.axvline(x = 3855.0+5400, color = 'm')
plt.show()
plt.title(' M2 IBI, S1 and S2')
plt.plot(M2_IB_stime[0],M2_IB[0],M2_IB_stime[1],M2_IB[1])
plt.xlabel('hours since start')
plt.xticks(x_h1,labels1)
plt.axvline(x = 2445.0, color = 'm')
plt.axvline(x = 2551.0+5400, color = 'm')
plt.show()
x_h2 = [0,3600,7200,10800,14400,18000,21600,25200]
labels2 = [0,1,2,3,4,5,6,7]
plt.xlabel('hours since start')
plt.xticks(x_h2,labels2)
plt.title(' F IBI, S1 and S2')
plt.plot(F_IB_stime[0],F_IB[0],F_IB_stime[1],F_IB[1])
plt.axvline(x = 963.0, color = 'm')
plt.axvline(x = 956.0+10800, color = 'm')
plt.show()

In [ ]:
plt.title(' M1 IBI, S3 and S4')
plt.plot(M1_IB_stime[2],M1_IB[2],M1_IB_stime[3],M1_IB[3])
plt.xlabel('hour since start')
x_h1 = [0,3600,7200,10800]
labels1 = [0,1,2,3]
plt.xticks(x_h1, labels1)
plt.axvline(x = 3792.0, color = 'm')
plt.axvline(x = 3821.0+5400, color = 'm')
plt.show()

plt.title(' M1 IBI, S5 and S6')
plt.plot(M1_IB_stime[4],M1_IB[4],M1_IB_stime[5],M1_IB[5])
plt.xlabel('hour since start')
plt.xticks(x_h1, labels1)
plt.axvline(x = 3879.0, color = 'm')
plt.axvline(x = 3767.0+5400, color = 'm')
plt.show()

plt.title(' M1 IBI, S7 and S8')
plt.plot(M1_IB_stime[6],M1_IB[6],M1_IB_stime[7],M1_IB[7])
plt.xlabel('hour since start')
plt.xticks(x_h1, labels1)
plt.axvline(x = 3793.0, color = 'm')
plt.axvline(x = 3820.0+5400, color = 'm')
plt.show()

plt.title(' M1 IBI, S9 and S10')
plt.plot(M1_IB_stime[8],M1_IB[8],M2_IB_stime[9],M2_IB[9])
plt.xlabel('hour since start')
plt.xticks(x_h1, labels1)
plt.axvline(x = 3835.0, color = 'm')
plt.axvline(x = 3875.0+5400, color = 'm')
plt.gca().legend(('student 9','student 10'))
plt.show()

In [ ]:
plt.title(' M2 IBI, S3 and S4')
plt.plot(M2_IB_stime[2],M2_IB[2],M2_IB_stime[3],M2_IB[3])
plt.xlabel('hour since start')
x_h1 = [0,3600,7200,10800]
labels1 = [0,1,2,3]
plt.xticks(x_h1, labels1)
plt.axvline(x = 2606.0, color = 'm')
plt.axvline(x = 2467.0+5400, color = 'm')
plt.gca().legend(('student 3','student 4'))
plt.show()

plt.title(' M2 IBI, S5 and S6')
plt.plot(M2_IB_stime[4],M2_IB[4],M2_IB_stime[5],M2_IB[5])
plt.xlabel('hour since start')
plt.xticks(x_h1, labels1)
plt.axvline(x = 2502.0, color = 'g')
plt.axvline(x = 2512.0+5400, color = 'm')
plt.show()

plt.title(' M2 IBI, S7 and S8')
plt.plot(M2_IB_stime[6],M2_IB[6],M2_IB_stime[7],M2_IB[7])
plt.xlabel('hour since start')
plt.xticks(x_h1, labels1)
plt.axvline(x = 2597.0, color = 'm')
plt.axvline(x = 2616.0+5400, color = 'm')
plt.gca().legend(('student 7','student 8'))
plt.show()

plt.title(' M2 IBI, S9 and S10')
plt.plot(M2_IB_stime[8],M2_IB[8],M2_IB_stime[9],M2_IB[9])
plt.xlabel('hour since start')
plt.xticks(x_h1, labels1)
plt.axvline(x = 2545.0, color = 'g')
plt.axvline(x = 2475.0+5400, color = 'm')
plt.show()

In [ ]:
plt.title(' F IBI, S5 and S6')
plt.plot(F_IB_stime[4],F_IB[4],F_IB_stime[5],F_IB[5])
plt.xlabel('hour since start')
x_h2 = [0,3600,7200,10800,14400,18000,21600,25200]
labels2 = [0,1,2,3,4,5,6,7]
plt.xticks(x_h2,labels2)
plt.axvline(x = 966.0, color = 'g')
plt.axvline(x = 953.0+10800, color = 'm')
plt.show()

In [ ]:
plt.title(' F IBI, S3 and S4')
plt.plot(F_IB_stime[2],F_IB[2],F_IB_stime[3],F_IB[3])
plt.xlabel('hour since start')
x_h2 = [0,3600,7200,10800,14400]
labels2 = [0,1,2,3,4]
plt.xticks(x_h2,labels2)
plt.axvline(x = 966.0, color = 'g')
plt.axvline(x = 958.0+10800, color = 'm')
plt.show()

plt.title(' F IBI, S7 and S8')
plt.plot(F_IB_stime[6],F_IB[6],F_IB_stime[7],F_IB[7])
plt.xlabel('hour since start')
plt.xticks(x_h2,labels2)
plt.axvline(x = 964.0, color = 'g')
plt.axvline(x = 961.0+10800, color = 'm')
plt.show()

plt.title(' F IBI, S9 and S10')
plt.plot(F_IB_stime[8],F_IB[8],F_IB_stime[9],F_IB[9])
plt.xlabel('hour since start')
plt.xticks(x_h2,labels2)
plt.axvline(x = 953.0, color = 'g')
plt.axvline(x = 966.0+10800, color = 'm')
plt.show()

In [ ]:
x1=[]
for i in range(len(M1_EDA[0])):
  x1.append(i/M1_EDA_fr[0]/3600)
x2=[]
for i in range(len(M1_EDA[1])):
  x2.append(i/M1_EDA_fr[1]/3600)
#x3=[]
#for i in range(len(M1_EDA[2])):
#  x3.append(i/M1_EDA_fr[2]/3600)

plt.plot(x1,M1_EDA[0],x2,M1_EDA[1]) #,x3,M1_EDA[2]
plt.title(' EDA M1 1 and 2')
plt.xlabel('hour since start')
plt.show()

In [ ]:
x1=[]
for i in range(len(M1_EDA[2])):
  x1.append(i/M1_EDA_fr[2]/3600)
x2=[]
for i in range(len(M1_EDA[3])):
  x2.append(i/M1_EDA_fr[3]/3600)

plt.plot(x1,M1_EDA[2],x2,M1_EDA[3])
plt.title('EDA M1 3 and 4')
plt.xlabel('hour since start')
plt.show()

x1=[]
for i in range(len(M1_EDA[4])):
  x1.append(i/M1_EDA_fr[4]/3600)
x2=[]
for i in range(len(M1_EDA[5])):
  x2.append(i/M1_EDA_fr[5]/3600)

plt.plot(x1,M1_EDA[4],x2,M1_EDA[5])
plt.title('EDA M1 5 and 6')
plt.xlabel('hour since start')
plt.show()

x1=[]
for i in range(len(M1_EDA[6])):
  x1.append(i/M1_EDA_fr[6]/3600)
x2=[]
for i in range(len(M1_EDA[7])):
  x2.append(i/M1_EDA_fr[7]/3600)

plt.plot(x1,M1_EDA[6],x2,M1_EDA[7])
plt.title('EDA M1 7 and 8')
plt.xlabel('hour since start')
plt.show()

x1=[]
for i in range(len(M1_EDA[8])):
  x1.append(i/M1_EDA_fr[8]/3600)
x2=[]
for i in range(len(M1_EDA[9])):
  x2.append(i/M1_EDA_fr[9]/3600)

plt.plot(x1,M1_EDA[8],x2,M1_EDA[9])
plt.title('EDA M1 9 and 10')
plt.xlabel('hour since start')
plt.show()

In [ ]:
x1=[]
for i in range(len(M2_EDA[0])):
  x1.append(i/M2_EDA_fr[0]/3600)
x2=[]
for i in range(len(M2_EDA[1])):
  x2.append(i/M2_EDA_fr[1]/3600)

plt.plot(x1,M2_EDA[0],x2,M2_EDA[1])
plt.title('EDA M2 1 and 2')
plt.xlabel('hour since start')
plt.show()

x1=[]
for i in range(len(M2_EDA[2])):
  x1.append(i/M2_EDA_fr[2]/3600)
x2=[]
for i in range(len(M2_EDA[3])):
  x2.append(i/M2_EDA_fr[3]/3600)

plt.plot(x1,M2_EDA[2],x2,M2_EDA[3])
plt.title('EDA M2 3 and 4')
plt.xlabel('hour since start')
plt.show()

x1=[]
for i in range(len(M2_EDA[4])):
  x1.append(i/M2_EDA_fr[4]/3600)
x2=[]
for i in range(len(M2_EDA[5])):
  x2.append(i/M2_EDA_fr[5]/3600)

plt.plot(x1,M2_EDA[4],x2,M2_EDA[5])
plt.title('EDA M2 5 and 6')
plt.xlabel('hour since start')
plt.show()

x1=[]
for i in range(len(M2_EDA[6])):
  x1.append(i/M2_EDA_fr[6]/3600)
x2=[]
for i in range(len(M2_EDA[7])):
  x2.append(i/M2_EDA_fr[7]/3600)

plt.plot(x1,M2_EDA[6],x2,M2_EDA[7])
plt.title('EDA M2 7 and 8')
plt.xlabel('hour since start')
plt.show()

x1=[]
for i in range(len(M2_EDA[8])):
  x1.append(i/M2_EDA_fr[8]/3600)
x2=[]
for i in range(len(M2_EDA[9])):
  x2.append(i/M2_EDA_fr[9]/3600)

plt.plot(x1,M2_EDA[8],x2,M2_EDA[9])
plt.title('EDA M2 9 and 10')
plt.xlabel('hour since start')
plt.show()

In [ ]:
x1=[]
for i in range(len(F_EDA[0])):
  x1.append(i/F_EDA_fr[0]/3600)
x2=[]
for i in range(len(F_EDA[1])):
  x2.append(i/F_EDA_fr[1]/3600)

plt.plot(x1,F_EDA[0],x2,F_EDA[1])
plt.title('EDA F 1 and 2')
plt.xlabel('hour since start')
plt.show()

x1=[]
for i in range(len(F_EDA[2])):
  x1.append(i/F_EDA_fr[2]/3600)
x2=[]
for i in range(len(F_EDA[3])):
  x2.append(i/F_EDA_fr[3]/3600)

plt.plot(x1,F_EDA[2],x2,F_EDA[3])
plt.title('EDA F 3 and 4')
plt.xlabel('hour since start')
plt.show()

x1=[]
for i in range(len(F_EDA[4])):
  x1.append(i/F_EDA_fr[4]/3600)
x2=[]
for i in range(len(F_EDA[5])):
  x2.append(i/F_EDA_fr[5]/3600)

plt.plot(x1,F_EDA[4],x2,F_EDA[5])
plt.title('EDA F 5 and 6')
plt.xlabel('hour since start')
plt.show()

x1=[]
for i in range(len(F_EDA[6])):
  x1.append(i/F_EDA_fr[6]/3600)
x2=[]
for i in range(len(F_EDA[7])):
  x2.append(i/F_EDA_fr[7]/3600)

plt.plot(x1,F_EDA[6],x2,F_EDA[7])
plt.title('EDA F 7 and 8')
plt.xlabel('hour since start')
plt.show()

x1=[]
for i in range(len(F_EDA[8])):
  x1.append(i/F_EDA_fr[8]/3600)
x2=[]
for i in range(len(F_EDA[9])):
  x2.append(i/F_EDA_fr[9]/3600)

plt.plot(x1,F_EDA[8],x2,F_EDA[9])
plt.title('EDA F 9 and 10')
plt.xlabel('hour since start')
plt.show()

In [ ]:
x1=[]
for i in range(len(M1_BVP[0])):
  x1.append(i/M1_BVP_fr[0]/3600)

x2=[]
for i in range(len(M1_BVP[1])):
  x2.append(i/M1_BVP_fr[1]/3600)

plt.plot(x1,M1_BVP[0],x2,M1_BVP[1])
plt.title(' BVP M1 S1 and S2')
plt.xlabel('hour since start')
#plt.xlim((1.35,0.5))
plt.show()
#print(M1_BVP[0][0:90])
#print(M1_BVP[1][0:90])
#print(M1_BVP[2][0:90])

In [ ]:
M1_BVP_ave=[]
t_M1_BVPa=[]
atime = 20 # sec 
M1_BVP_ave, t_M1_BVP = smooth_data(M1_BVP,M1_BVP_fr,atime)

plt.plot(x1,M1_BVP[0],x2,M1_BVP[1])
plt.title(' BVP M1 1 and 2')
plt.xlabel('hour since start')
plt.xlim((-0.0001,0.002))
plt.show()
plt.plot(x1,M1_BVP[0],x2,M1_BVP[1])
plt.xlim((1.015,1.025))
plt.ylim((-120,190))
plt.show()
plt.plot(t_M1_BVP[0],M1_BVP_ave[0],t_M1_BVP[1],M1_BVP_ave[1])
plt.xlim((.0,3.1))
#plt.ylim((-120,190))
plt.title(' smooth BVP M1 1 and 2')
plt.show()

In [ ]:
plt.plot(t_M1_BVP[2],M1_BVP_ave[2],t_M1_BVP[3],M1_BVP_ave[3])
plt.xlim((.0,3.1))
#plt.ylim((-120,190))
plt.title(' smooth BVP M1 3 and 4')
plt.show()

plt.plot(t_M1_BVP[4],M1_BVP_ave[4],t_M1_BVP[5],M1_BVP_ave[5])
plt.xlim((.0,3.1))
#plt.ylim((-120,190))
plt.title(' smooth BVP M1 5 and 6')
plt.show()

plt.plot(t_M1_BVP[6],M1_BVP_ave[6],t_M1_BVP[7],M1_BVP_ave[7])
plt.xlim((.0,3.1))
#plt.ylim((-120,190))
plt.title(' smooth BVP M1 7 and 8')
plt.show()

plt.plot(t_M1_BVP[8],M1_BVP_ave[8],t_M1_BVP[9],M1_BVP_ave[9])
plt.xlim((.0,3.1))
#plt.ylim((-120,190))
plt.title(' smooth BVP M1 9 and 10')
plt.show()

In [ ]:
M2_BVP_ave=[]
t_M2_BVP=[]
atime = 20 # sec 
M2_BVP_ave, t_M2_BVP = smooth_data(M2_BVP,M2_BVP_fr,atime)

plt.plot(t_M2_BVP[0],M2_BVP_ave[0],t_M2_BVP[1],M2_BVP_ave[1])
plt.xlim((.0,3.1))
#plt.ylim((-120,190))
plt.title(' smooth BVP M2 1 and 2')
plt.show()

plt.plot(t_M2_BVP[2],M2_BVP_ave[2],t_M2_BVP[3],M2_BVP_ave[3])
plt.xlim((.0,3.1))
#plt.ylim((-120,190))
plt.title(' smooth BVP M2 3 and 4')
plt.show()

plt.plot(t_M2_BVP[4],M2_BVP_ave[4],t_M2_BVP[5],M2_BVP_ave[5])
plt.xlim((.0,3.1))
#plt.ylim((-120,190))
plt.title(' smooth BVP M2 5 and 6')
plt.show()

plt.plot(t_M2_BVP[6],M2_BVP_ave[6],t_M2_BVP[7],M2_BVP_ave[7])
plt.xlim((.0,3.1))
#plt.ylim((-120,190))
plt.title(' smooth BVP M2 7 and 8')
plt.show()

plt.plot(t_M2_BVP[8],M2_BVP_ave[8],t_M2_BVP[9],M2_BVP_ave[9])
plt.xlim((.0,3.1))
#plt.ylim((-120,190))
plt.title(' smooth BVP M2 9 and 10')
plt.show()

In [ ]:
F_BVP_ave=[]
t_F_BVP=[]
atime = 20 # sec 
F_BVP_ave, t_F_BVP = smooth_data(F_BVP,F_BVP_fr,atime)

plt.plot(t_F_BVP[0],F_BVP_ave[0],t_F_BVP[1],F_BVP_ave[1])
plt.xlim((.0,3.1))
#plt.ylim((-120,190))
plt.title(' smooth BVP F 1 and 2')
plt.show()

plt.plot(t_F_BVP[2],F_BVP_ave[2],t_F_BVP[3],F_BVP_ave[3])
plt.xlim((.0,3.1))
#plt.ylim((-120,190))
plt.title(' smooth BVP F 3 and 4')
plt.show()

plt.plot(t_F_BVP[4],F_BVP_ave[4],t_F_BVP[5],F_BVP_ave[5])
plt.xlim((.0,3.1))
#plt.ylim((-120,190))
plt.title(' smooth BVP F 5 and 6')
plt.show()

plt.plot(t_F_BVP[6],F_BVP_ave[6],t_F_BVP[7],F_BVP_ave[7])
plt.xlim((.0,3.1))
#plt.ylim((-120,190))
plt.title(' smooth BVP F 7 and 8')
plt.show()

plt.plot(t_F_BVP[8],F_BVP_ave[8],t_F_BVP[9],F_BVP_ave[9])
plt.xlim((.0,3.1))
#plt.ylim((-120,190))
plt.title(' smooth BVP F 9 and 10')
plt.show()

From here on we calculate the r^2 value of two different biosignals.

In [ ]:
plt.plot(M1_TE_mean_ex,M1_HR_mean_ex,'o',M2_TE_mean_ex,M2_HR_mean_ex,'o',F_TE_mean_ex,F_HR_mean_ex,'o')
plt.title(' Temp exam vs heart rate, blue M1, orange M2, Green Final')
plt.ylabel('heart rate')
plt.show()
linfit = linregress(M1_TE_mean_ex, M1_HR_mean_ex)
r=linfit.rvalue
print('M1 Temp vs HR  R^2:', r*r)
linfit = linregress(M2_TE_mean_ex, M2_HR_mean_ex)
r=linfit.rvalue
print('M2 Temp vs HR  R^2:', r*r)
linfit = linregress(F_TE_mean_ex, F_HR_mean_ex)
r=linfit.rvalue
print('F  Temp vs HR  R^2:', r*r)

In [ ]:
plt.plot(M1_BVP_mean_ex,M1_HR_mean_ex,'o',M2_BVP_mean_ex,M2_HR_mean_ex,'o',F_BVP_mean_ex,F_HR_mean_ex,'o')
plt.title(' BVP exam vs heart rate, blue M1, orange M2, Green Final')
plt.ylabel('heart rate')
plt.show()
linfit = linregress(M1_BVP_mean_ex, M1_HR_mean_ex)
r=linfit.rvalue
print('M1 BVP vs HR  R^2:', r*r)
linfit = linregress(M2_BVP_mean_ex, M2_HR_mean_ex)
r=linfit.rvalue
print('M2 BVP vs HR  R^2:', r*r)
linfit = linregress(F_BVP_mean_ex, F_HR_mean_ex)
r=linfit.rvalue
print('F  BVP vs HR  R^2:', r*r)

In [ ]:
plt.plot(M1_IB_mean_ex,M1_HR_mean_ex,'o',M2_IB_mean_ex,M2_HR_mean_ex,'o',F_IB_mean_ex,F_HR_mean_ex,'o')
plt.title(' IBI exam vs heart rate, blue M1, orange M2, Green Final')
plt.ylabel('heart rate')
plt.show()
linfit = linregress(M1_IB_mean_ex, M1_HR_mean_ex)
r=linfit.rvalue
print('M1 IBI vs HR  R^2:', r*r)
linfit = linregress(M2_IB_mean_ex, M2_HR_mean_ex)
r=linfit.rvalue
print('M2 IBI vs HR  R^2:', r*r)
linfit = linregress(F_IB_mean_ex, F_HR_mean_ex)
r=linfit.rvalue
print('F  IBI vs HR  R^2:', r*r)

In [ ]:
plt.plot(M1_EDA_mean_ex,M1_HR_mean_ex,'o',M2_EDA_mean_ex,M2_HR_mean_ex,'o',F_EDA_mean_ex,F_HR_mean_ex,'o')
plt.title(' EDA exam vs heart rate, blue M1, orange M2, Green Final')
plt.ylabel('heart rate')
plt.show()
linfit = linregress(M1_EDA_mean_ex, M1_HR_mean_ex)
r=linfit.rvalue
print('M1 EDA vs HR  R^2:', r*r)
linfit = linregress(M2_EDA_mean_ex, M2_HR_mean_ex)
r=linfit.rvalue
print('M2 EDA vs HR  R^2:', r*r)
linfit = linregress(F_EDA_mean_ex, F_HR_mean_ex)
r=linfit.rvalue
print('F  EDA vs HR  R^2:', r*r)

In [ ]:
plt.plot(M1_EDA_mean_ex,M1_IB_mean_ex,'o',M2_EDA_mean_ex,M2_IB_mean_ex,'o',F_EDA_mean_ex,F_IB_mean_ex,'o')
plt.title(' EDA exam vs IBI, blue M1, orange M2, Green Final')
plt.ylabel('IBI')
plt.show()
linfit = linregress(M1_EDA_mean_ex, M1_IB_mean_ex)
r=linfit.rvalue
print('M1 EDA vs IBI  R^2:', r*r)
linfit = linregress(M2_EDA_mean_ex, M2_IB_mean_ex)
r=linfit.rvalue
print('M2 EDA vs IBI  R^2:', r*r)
linfit = linregress(F_EDA_mean_ex, F_IB_mean_ex)
r=linfit.rvalue
print('F  EDA vs IBI  R^2:', r*r)

In [ ]:
plt.plot(M1_TE_mean_ex,M1_IB_mean_ex,'o',M2_TE_mean_ex,M2_IB_mean_ex,'o',F_TE_mean_ex,F_IB_mean_ex,'o')
plt.title(' TEMP exam vs IBI, blue M1, orange M2, Green Final')
plt.ylabel('IBI')
plt.show()
linfit = linregress(M1_TE_mean_ex, M1_IB_mean_ex)
r=linfit.rvalue
print('M1 TEMP vs IBI  R^2:', r*r)
linfit = linregress(M2_TE_mean_ex, M2_IB_mean_ex)
r=linfit.rvalue
print('M2 TEMP vs IBI  R^2:', r*r)
linfit = linregress(F_TE_mean_ex, F_IB_mean_ex)
r=linfit.rvalue
print('F  TEMP vs IBI  R^2:', r*r)

In [ ]:
plt.plot(M1_BVP_mean_ex,M1_IB_mean_ex,'o',M2_BVP_mean_ex,M2_IB_mean_ex,'o',F_BVP_mean_ex,F_IB_mean_ex,'o')
plt.title(' BVP exam vs IBI, blue M1, orange M2, Green Final')
plt.ylabel('IBI')
plt.show()
linfit = linregress(M1_BVP_mean_ex, M1_IB_mean_ex)
r=linfit.rvalue
print('M1 BVP vs IBI  R^2:', r*r)
linfit = linregress(M2_BVP_mean_ex, M2_IB_mean_ex)
r=linfit.rvalue
print('M2 BVP vs IBI  R^2:', r*r)
linfit = linregress(F_BVP_mean_ex, F_IB_mean_ex)
r=linfit.rvalue
print('F  BVP vs IBI  R^2:', r*r)

In [ ]:
plt.plot(M1_BVP_mean_ex,M1_TE_mean_ex,'o',M2_BVP_mean_ex,M2_TE_mean_ex,'o',F_BVP_mean_ex,F_TE_mean_ex,'o')
plt.title(' BVP exam vs TEMP, blue M1, orange M2, Green Final')
plt.ylabel('TEMP')
plt.show()
linfit = linregress(M1_BVP_mean_ex, M1_TE_mean_ex)
r=linfit.rvalue
print('M1 BVP vs TEMP  R^2:', r*r)
linfit = linregress(M2_BVP_mean_ex, M2_TE_mean_ex)
r=linfit.rvalue
print('M2 BVP vs TEMP  R^2:', r*r)
linfit = linregress(F_BVP_mean_ex, F_TE_mean_ex)
r=linfit.rvalue
print('F  BVP vs TEMP  R^2:', r*r)

In [ ]:
plt.plot(M1_EDA_mean_ex,M1_TE_mean_ex,'o',M2_EDA_mean_ex,M2_TE_mean_ex,'o',F_EDA_mean_ex,F_TE_mean_ex,'o')
plt.title(' EDA exam vs TEMP, blue M1, orange M2, Green Final')
plt.ylabel('TEMP')
plt.show()
linfit = linregress(M1_EDA_mean_ex, M1_TE_mean_ex)
r=linfit.rvalue
print('M1 EDA vs TEMP  R^2:', r*r)
linfit = linregress(M2_EDA_mean_ex, M2_TE_mean_ex)
r=linfit.rvalue
print('M2 EDA vs TEMP  R^2:', r*r)
linfit = linregress(F_EDA_mean_ex, F_TE_mean_ex)
r=linfit.rvalue
print('F  EDA vs TEMP  R^2:', r*r)

In [ ]:
plt.plot(M1_EDA_mean_ex,M1_BVP_mean_ex,'o',M2_EDA_mean_ex,M2_BVP_mean_ex,'o',F_EDA_mean_ex,F_BVP_mean_ex,'o')
plt.title(' EDA exam vs BVP, blue M1, orange M2, Green Final')
plt.ylabel('BVP')
plt.show()
linfit = linregress(M1_EDA_mean_ex, M1_BVP_mean_ex)
r=linfit.rvalue
print('M1 EDA vs BVP  R^2:', r*r)
linfit = linregress(M2_EDA_mean_ex, M2_BVP_mean_ex)
r=linfit.rvalue
print('M2 EDA vs BVP  R^2:', r*r)
linfit = linregress(F_EDA_mean_ex, F_BVP_mean_ex)
r=linfit.rvalue
print('F  EDA vs BVP  R^2:', r*r)

In [ ]:
plt.plot(M1_TE_mean_ex,Grades[:,0],'o',M2_TE_mean_ex,Grades[:,1],'o',F_TE_mean_ex,Grade2[:,2],'o')
#plt.plot(M1_TE_mean,Grades[:,0],'+',M2_TE_mean,Grades[:,1],'+')
plt.title(' Temp vs grades, blue M1, orange M2, Green Final')
plt.ylabel('grade')
plt.show()

Y=range(25,35)
linfit = linregress(M1_TE_mean_ex, Grades[:,0])
print('General accuracy of M1 TE Grade data:\n')
a=linfit.slope
b=linfit.intercept
r=linfit.rvalue
print('y = {}X + {}'.format(a, b))
print('R^2:', r*r)
fit=a*Y+b
plt.plot(Y,fit,M1_TE_mean_ex,Grades[:,0],'o')
plt.show()
linfit = linregress(M2_TE_mean_ex, Grades[:,1])
print('General accuracy of M2 TE Grade data:\n')
a=linfit.slope
b=linfit.intercept
r=linfit.rvalue
print('y = {}X + {}'.format(linfit.slope, linfit.intercept))
print('R^2:', r*r)
fit=a*Y+b
plt.plot(Y,fit,M2_TE_mean_ex,Grades[:,1],'o')
plt.show()

In [ ]:
plt.plot(M1_HR_mean_ex,Grades[:,0],'o',M2_HR_mean_ex,Grades[:,1],'o',F_HR_mean_ex,Grade2[:,2],'o')
plt.title(' Heart rate in exam vs grades, blue M1, orange M2, Green Final')
plt.ylabel('grade')
plt.show()
Y=range(75,110)
linfit = linregress(M1_HR_mean_ex, Grades[:,0])
print('General accuracy of M1 HR Grade data:\n')
a=linfit.slope
b=linfit.intercept
r=linfit.rvalue
print('y = {}X + {}'.format(linfit.slope, linfit.intercept))
print('R^2:', r*r)
fit=a*Y+b
plt.plot(Y,fit,M1_HR_mean_ex,Grades[:,0],'o')
plt.show()
linfit = linregress(M2_HR_mean_ex, Grades[:,1])
print('General accuracy of M2 HR Grade data:\n')
a=linfit.slope
b=linfit.intercept
r=linfit.rvalue
print('y = {}X + {}'.format(linfit.slope, linfit.intercept))
print('R^2:', r*r)
fit=a*Y+b
plt.plot(Y,fit,M2_HR_mean_ex,Grades[:,1],'o')
plt.show()
linfit = linregress(F_HR_mean_ex, Grade2[:,2])
print('General accuracy of F HR Grade data:\n')
a=linfit.slope
b=linfit.intercept
r=linfit.rvalue
print('y = {}X + {}'.format(a, b))
print('R^2:', r*r)
fit=a*Y+b
plt.plot(Y,fit,F_HR_mean_ex,Grade2[:,2],'o')
plt.show()

In [ ]:
plt.plot(M1_BVP_mean_ex,Grades[:,0],'o',M2_BVP_mean_ex,Grades[:,1],'o',F_BVP_mean_ex,Grade2[:,2],'o')
plt.title(' BVP in exam vs grades, blue M1, orange M2, Green Final')
plt.ylabel('grade')
plt.show()

linfit = linregress(M1_BVP_mean_ex, Grades[:,1])
print('General accuracy of M1 BVP Grade data:\n')
r=linfit.rvalue
print('R^2:', r*r)
linfit = linregress(M2_BVP_mean_ex, Grades[:,1])
print('General accuracy of M2 BVP Grade data:\n')
r=linfit.rvalue
print('R^2:', r*r)
linfit = linregress(F_BVP_mean_ex, Grade2[:,2])
print('General accuracy of F BVP Grade data:\n')
r=linfit.rvalue
print('R^2:', r*r)

In [ ]:
plt.plot(M1_IB_mean_ex,Grades[:,0],'o',M2_IB_mean_ex,Grades[:,1],'o',F_IB_mean_ex,Grade2[:,2],'o')
plt.title(' IBI in exam vs grades, blue M1, orange M2, Green Final')
plt.ylabel('grade')
plt.show()

linfit = linregress(M1_IB_mean_ex, Grades[:,1])
print('General accuracy of M1 IBI Grade data:\n')
r=linfit.rvalue
print('R^2:', r*r)
linfit = linregress(M2_IB_mean_ex, Grades[:,1])
print('General accuracy of M2 IBI Grade data:\n')
r=linfit.rvalue
print('R^2:', r*r)
linfit = linregress(F_IB_mean_ex, Grade2[:,2])
print('General accuracy of F IBI Grade data:\n')
r=linfit.rvalue
print('R^2:', r*r)

In [ ]:
plt.plot(M1_EDA_mean_ex,Grades[:,0],'o',M2_EDA_mean_ex,Grades[:,1],'o',F_EDA_mean_ex,Grade2[:,2],'o')
plt.title(' EDA in exam vs grades, blue M1, orange M2, Green Final')
plt.ylabel('grade')
plt.show()

linfit = linregress(M1_EDA_mean_ex, Grades[:,1])
print('General accuracy of M1 EDA Grade data:\n')
r=linfit.rvalue
print('R^2:', r*r)
linfit = linregress(M2_EDA_mean_ex, Grades[:,1])
print('General accuracy of M2 EDA Grade data:\n')
r=linfit.rvalue
print('R^2:', r*r)
linfit = linregress(F_EDA_mean_ex, Grade2[:,2])
print('General accuracy of F EDA Grade data:\n')
r=linfit.rvalue
print('R^2:', r*r)

In [ ]:
linfit = linregress(M2_HR_mean_af, Grades[:,1])
r=linfit.rvalue
print('M2 HR after R^2:', r*r)
linfit = linregress(M2_HR_mean, Grades[:,1])
r=linfit.rvalue
print('M2 HR all R^2:', r*r)
linfit = linregress(M2_HR_mean_ex, Grades[:,1])
r=linfit.rvalue
print('M2 HR exam R^2:', r*r)

In [ ]:
plt.plot(M1_EDA_mean_ex,Grades[:,0],'o',M2_EDA_mean_ex,Grades[:,1],'o',F_EDA_mean_ex,Grade2[:,2],'o')
plt.title(' EDA vs grades, blue M1, orange M2, Green Final')
plt.ylabel('grade')
plt.show()
linfit = linregress(M1_EDA_mean_ex, Grades[:,0])
r=linfit.rvalue
print('M1 EDA vs Grade  R^2:', r*r)
linfit = linregress(M2_EDA_mean_ex, Grades[:,1])
r=linfit.rvalue
print('M2 EDA vs grade  R^2:', r*r)
linfit = linregress(F_EDA_mean_ex, Grade2[:,2])
r=linfit.rvalue
print('F  EDA vs grade  R^2:', r*r)
plt.plot(M1_HR_mean_ex,Grades[:,0],'o',M2_HR_mean_ex,Grades[:,1],'o',F_HR_mean_ex,Grade2[:,2],'o')
plt.title(' HR vs grades, blue M1, orange M2, Green Final')
plt.ylabel('grade')
plt.show()
linfit = linregress(M1_HR_mean_ex, Grades[:,0])
r=linfit.rvalue
print('M1 HR vs Grade  R^2:', r*r)
linfit = linregress(M2_HR_mean_ex, Grades[:,1])
r=linfit.rvalue
print('M2 HR vs grade  R^2:', r*r)
linfit = linregress(F_HR_mean_ex, Grade2[:,2])
r=linfit.rvalue
print('F  HR vs grade  R^2:', r*r)

In [ ]:
plt.plot(M1_IB_mean_ex,Grades[:,0],'o',M2_IB_mean_ex,Grades[:,1],'o',F_IB_mean_ex,Grade2[:,2],'o')
plt.title(' IBI vs grades, blue M1, orange M2, Green Final')
plt.ylabel('grade')
plt.show()
linfit = linregress(M1_IB_mean_ex, Grades[:,0])
r=linfit.rvalue
print('M1 IBI vs Grade  R^2:', r*r)
linfit = linregress(M2_IB_mean_ex, Grades[:,1])
r=linfit.rvalue
print('M2 IBI vs grade  R^2:', r*r)
linfit = linregress(F_IB_mean_ex, Grade2[:,2])
r=linfit.rvalue
print('F  IBI vs grade  R^2:', r*r)

plt.plot(M1_BVP_mean_ex,Grades[:,0],'o',M2_BVP_mean_ex,Grades[:,1],'o',F_BVP_mean_ex,Grade2[:,2],'o')
plt.title(' BVP vs grades, blue M1, orange M2, Green Final')
plt.ylabel('grade')
plt.show()
linfit = linregress(M1_BVP_mean_ex, Grades[:,0])
r=linfit.rvalue
print('M1 BVP vs Grade  R^2:', r*r)
linfit = linregress(M2_BVP_mean_ex, Grades[:,1])
r=linfit.rvalue
print('M2 BVP vs grade  R^2:', r*r)
linfit = linregress(F_BVP_mean_ex, Grade2[:,2])
r=linfit.rvalue
print('F  BVP vs grade  R^2:', r*r)

In [ ]:
plt.plot(M1_TE_mean_ex,Grades[:,0],'o',M2_TE_mean_ex,Grades[:,1],'o',F_TE_mean_ex,Grade2[:,2],'o')
plt.title(' TEMP vs grades, blue M1, orange M2, Green Final')
plt.ylabel('grade')
plt.show()
linfit = linregress(M1_TE_mean_ex, Grades[:,0])
r=linfit.rvalue
print('M1 TEMP vs Grade  R^2:', r*r)
linfit = linregress(M2_TE_mean_ex, Grades[:,1])
r=linfit.rvalue
print('M2 TEMP vs grade  R^2:', r*r)
linfit = linregress(F_TE_mean_ex, Grade2[:,2])
r=linfit.rvalue
print('F  TEMP vs grade  R^2:', r*r)

From here on, we start lengthening the data to be as long as BVP data.

In [ ]:
for i in range(len(M1_EDA)):
  print('studet',i,'points',len(M1_EDA[i]),' freq',M1_EDA_fr[i],'HZ, total time',len(M1_EDA[i])/M1_EDA_fr[i],
        's, start time ',datetime.datetime.fromtimestamp(M1_EDA_time[i]))

for i in range(len(M1_BVP)):
  print('student',i,'points',len(M1_BVP[i]),' freq',M1_BVP_fr[i],'HZ, total time',len(M1_BVP[i])/M1_BVP_fr[i],
        's,  start time ',datetime.datetime.fromtimestamp(M1_BVP_time[i]))

In [ ]:
M1_EDA_long=[]
M2_EDA_long=[]
F_EDA_long=[]
for exam in ['M1','M2','F']:
    if exam == 'M1':
        bvp_name = M1_BVP
        eda_name = M1_EDA
        long_name = M1_EDA_long
    if exam == 'M2':
        bvp_name = M2_BVP
        eda_name = M2_EDA
        long_name = M2_EDA_long
    if exam == 'F':
        bvp_name = F_BVP
        eda_name = F_EDA
        long_name = F_EDA_long
    for i in range(len(bvp_name)):
      arr_long=[]
      for k in range(len(bvp_name[i])):
        kslow=min(int(M1_EDA_fr[i]*k/64),len(eda_name[i])-1)  # goes 16 times slower, if k/16 goes over take the last point
        arr_long.append(eda_name[i][kslow])
      print(len(arr_long), 'EDA should be same as BVP len, student',i,exam)
      long_name.append(arr_long)

    
M1_HR_long=[]
M2_HR_long=[]
F_HR_long=[]
for exam in ['M1','M2','F']:
    if exam == 'M1':
        bvp_name = M1_BVP
        hr_name = M1_HR
        long_name =M1_HR_long
    if exam == 'M2':
        bvp_name = M2_BVP
        hr_name = M2_HR
        long_name = M2_HR_long
    if exam == 'F':
        bvp_name = F_BVP
        hr_name = F_HR
        long_name = F_HR_long
    for i in range(len(bvp_name)):
        arr_long=[]
        for k in range(len(bvp_name[i])):
            kslow=min(int(M1_HR_fr[i]*k/64),len(hr_name[i])-1)  # goes 64 times slower, if k/16 goes over take the last point
            arr_long.append(hr_name[i][kslow])
        print(len(arr_long), 'HR should be same as BVP len, student',i,exam)
        long_name.append(arr_long)
    
    
M1_TE_long=[]
M2_TE_long=[]
F_TE_long=[]
for exam in ['M1','M2','F']:
    if exam == 'M1':
        bvp_name = M1_BVP
        te_name = M1_TE
        long_name = M1_TE_long
    if exam == 'M2':
        bvp_name = M2_BVP
        te_name = M2_TE
        long_name = M2_TE_long
    if exam == 'F':
        bvp_name = F_BVP
        te_name = F_TE
        long_name = F_TE_long
    for i in range(len(bvp_name)):
        arr_long=[]
        for k in range(len(bvp_name[i])):
            kslow=min(int(M1_TE_fr[i]*k/64),len(te_name[i])-1)  # goes 16 times slower, if k/16 goes over take the last point
            arr_long.append(te_name[i][kslow])
        print(len(arr_long), 'TE should be same as BVP len, student',i,exam)
        long_name.append(arr_long)

In [ ]:
M1_IBI_long=[]
M2_IBI_long=[]
F_IBI_long=[]
for exam in ['M1','M2','F']:
    if exam == 'M1':
        bvp_name = M1_BVP
        ibi_name = M1_IB
        ibi_stime = M1_IB_stime
    if exam == 'M2':
        bvp_name = M2_BVP
        ibi_name = M2_IB
        ibi_stime = M2_IB_stime
    if exam == 'F':
        bvp_name = F_BVP
        ibi_name = F_IB
        ibi_stime = F_IB_stime
    for i in range(10):
        arr_long=[]
        it=0
        for k in range(len(bvp_name[i])):     # k on bvp pituus
            bvp_time=k/64
            if it == 0:
                if bvp_time < ibi_stime[i][it]:
                    arr_long.append(0)
                elif bvp_time >= ibi_stime[i][it]:
                    arr_long.append(ibi_name[i][it])
                    it=it+1
            else:
                if bvp_time < ibi_stime[i][it]:
                    arr_long.append(arr_long[-1])
                elif bvp_time >= ibi_stime[i][it]:
                    if it < len(ibi_name[i])-1:
                        arr_long.append(ibi_name[i][it])
                        it=it+1
                        #print(it,bvp_time)
                    else:
                        arr_long.append(0)
                
        print(len(arr_long), 'IBI should be same as BVP len, student',i,exam)
        #print('arr long 25563',arr_long[25563:25566],ibi_stime[0][155])
        if exam == 'M1':
            M1_IBI_long.append(arr_long)
            #print('M1',len(M1_IBI_long))
        if exam == 'M2':
            M2_IBI_long.append(arr_long)
            #print('M2',len(M2_IBI_long))
        if exam == 'F':
            F_IBI_long.append(arr_long)
            #print('F',len(F_IBI_long))


In [ ]:
#test that midterm exam 1 HR and HR long are similar
x1=[]
for i in range(len(M1_HR[0])):
  x1.append(i/M1_HR_fr[0]/3600)

x2=[]
for i in range(len(M1_HR_long[0])):
  x2.append(i/M1_BVP_fr[0]/3600)

plt.plot(x1,M1_HR[0],x2,M1_HR_long[0])
plt.title(' HR M1 1 and 1 long')
plt.xlabel('hour since start')
plt.show()

From here on we create a new file with the long data and then limit the data to the lenght of the exams.

In [ ]:
import csv

for stud in range(10):
    x2=[]
    for i in range(len(M1_EDA_long[stud])):
      x2.append(i/M1_BVP_fr[stud])

    with open('S'+str(stud)+'_M1_2.csv', 'w', newline='') as csvfile:
        dataw = csv.writer(csvfile)
        dataw.writerow(['time in s', 'BVP', 'EDA', 'HR', 'TE', 'IBI' ])
        for i in range(len(M1_BVP[stud])):
           dataw.writerow([x2[i],M1_BVP[stud][i],M1_EDA_long[stud][i],M1_HR_long[stud][i],M1_TE_long[stud][i],M1_IBI_long[stud][i]])
    print(len(M1_BVP[stud]),' lines written to file S'+str(stud)+'_M1_2.csv')
    
    x2=[]
    for i in range(len(M2_EDA_long[stud])):
      x2.append(i/M2_BVP_fr[stud])

    with open('S'+str(stud)+'_M2_2.csv', 'w', newline='') as csvfile:
        dataw = csv.writer(csvfile)
        dataw.writerow(['time in s', 'BVP', 'EDA', 'HR', 'TE', 'IBI' ])
        for i in range(len(M2_BVP[stud])):
           dataw.writerow([x2[i],M2_BVP[stud][i],M2_EDA_long[stud][i],M2_HR_long[stud][i],M2_TE_long[stud][i],M2_IBI_long[stud][i]])
    print(len(M2_BVP[stud]),' lines written to file S'+str(stud)+'_M2_2.csv')
    
    x2=[]
    for i in range(len(F_EDA_long[stud])):
      x2.append(i/F_BVP_fr[stud])

    with open('S'+str(stud)+'_F_2.csv', 'w', newline='') as csvfile:
        dataw = csv.writer(csvfile)
        dataw.writerow(['time in s', 'BVP', 'EDA', 'HR', 'TE', 'IBI' ])
        for i in range(len(F_BVP[stud])):
           dataw.writerow([x2[i],F_BVP[stud][i],F_EDA_long[stud][i],F_HR_long[stud][i],F_TE_long[stud][i],F_IBI_long[stud][i]])
    print(len(F_BVP[stud]),' lines written to file S'+str(stud)+'_F_2.csv')
    

In [ ]:
ex_start_s_M1=[]
for i in range(10):
    start_sec = unix_M1-M1_TE_time[i]
    ex_start_s_M1.append(start_sec)
print('exam start time in seconds for M1',ex_start_s_M1)

ex_start_s_M2=[]
for i in range(10):
    start_sec = unix_M2-M2_TE_time[i]
    ex_start_s_M2.append(start_sec)
print('exam start time in seconds for M2',ex_start_s_M2)

ex_start_s_F=[]
for i in range(10):
    start_sec = unix_F-F_TE_time[i]
    ex_start_s_F.append(start_sec)
print('exam start time in seconds for F',ex_start_s_F)

In [ ]:
ex_end_s_M1=[]
for i in range(10):
    end_sec = ex_start_s_M1[i] + 5400
    ex_end_s_M1.append(end_sec)
print('exam end time in seconds for M1', ex_end_s_M1)

ex_end_s_M2=[]
for i in range(10):
    end_sec = ex_start_s_M2[i] + 5400
    ex_end_s_M2.append(end_sec)
print('exam end time in seconds for M2', ex_end_s_M2)

ex_end_s_F=[]
for i in range(10):
    end_sec = ex_start_s_F[i] + 5400*2
    ex_end_s_F.append(end_sec)
print('exam end time in seconds for F', ex_end_s_F)

In [ ]:
import csv
for exam in ['M1','M2','F']:
    for stud in range(10):
        lines = list()
        with open('S'+str(stud)+'_'+str(exam)+'_2.csv', 'r', newline='') as file:
            reader = csv.reader(file, delimiter=',')
            next(reader) #hyppää ensimmäisen rivin yli
            for row in reader:
                if exam == 'M1':
                    if (float(row[0])>=ex_start_s_M1[stud]) and (float(row[0])<=ex_end_s_M1[stud]):
                        lines.append(row)
                if exam == 'M2':
                    if (float(row[0])>=ex_start_s_M2[stud]) and (float(row[0])<=ex_end_s_M2[stud]):
                        lines.append(row)
                if exam == 'F':
                    if (float(row[0])>=ex_start_s_F[stud]) and (float(row[0])<=ex_end_s_F[stud]):
                        lines.append(row)


        with open('exam_S'+str(stud)+'_'+str(exam)+'.csv', 'w', newline='') as write_file:
            writer = csv.writer(write_file)
            writer.writerow(['time in s', 'BVP', 'EDA', 'HR', 'TE', 'IBI' ])
            writer.writerows(lines)
        print('new file written exam_S'+str(stud)+'_'+str(exam)+'.csv')

In [ ]:
#compare the values for smooth vs long vs raw
M1_HR_long_bf=[]
M1_HR_long_ex=[]
M1_HR_long_af=[]
M1_HR_frl = []
M1_HR_varl_ex = []
for i in range(10):
    M1_HR_frl.append(M1_HR_fr[i]*64)
for i in range(10):
    t0 = 0
    t1 = unix_M1-M1_HR_time[i]
    M1_HR_long_bf, M1_HR_stdl_bf, M1_HR_varl_bf = aver_data(M1_HR_long,M1_HR_frl,t0,t1) # HR before M1 exam
    t2 = t1 + 5400
    M1_HR_long_ex, M1_HR_stdl_ex, M1_HR_varl_ex = aver_data(M1_HR_long,M1_HR_frl,t1,t2) # HR during M1 exam
    t3 = int(len(M1_HR[i]))
    M1_HR_long_af, M1_HR_stdl_af, M1_HR_varl_ex = aver_data(M1_HR_long,M1_HR_frl,t2,t3) # HR after M1 exam
for i in range(10):
    M1_HR_varl_ex.append(M1_HR_stdl_ex[i]**2)
print('Heart rate smooth mean BEFORE ',EX,' exam \n', M1_HR_smo_bf, np.mean(M1_HR_smo_bf))
print('Heart rate smooth std BEFORE ',EX,' exam \n', M1_HR_stds_bf, np.mean(M1_HR_stds_bf))
print()
print('Heart rate mean BEFORE ',EX,' exam \n', M1_HR_mean_bf, np.mean(M1_HR_mean_bf))
print('Heart rate std BEFORE ',EX,' exam \n', M1_HR_std_bf, np.mean(M1_HR_std_bf))
print()
print('Heart rate long mean BEFORE ',EX,' exam \n', M1_HR_long_bf, np.mean(M1_HR_long_bf))
print('Heart rate long std BEFORE ',EX,' exam \n', M1_HR_stdl_bf, np.mean(M1_HR_stdl_bf))
print()
print('Heart rate smooth mean DURING ',EX,' exam \n', M1_HR_smo_ex, np.mean(M1_HR_smo_ex))
print('Heart rate smooth std DURING ',EX,' exam \n', M1_HR_stds_ex, np.mean(M1_HR_stds_ex))
print()
print('Heart rate DURING mean ',EX,' exam \n', M1_HR_mean_ex, np.mean(M1_HR_mean_ex))
print('Heart rate std DURING ',EX,' exam \n', M1_HR_std_ex, np.mean(M1_HR_std_ex))
print()
print('Heart rate long mean DURING ',EX,' exam \n', M1_HR_long_ex, np.mean(M1_HR_long_ex))
print('Heart rate long var DURING ',EX,' exam \n', M1_HR_varl_ex, np.mean(M1_HR_varl_ex))
print()
print('Heart rate smooth mean AFTER ',EX,' exam \n', M1_HR_smo_af, np.mean(M1_HR_smo_af))
print('Heart rate smooth std AFTER ',EX,' exam \n', M1_HR_stds_af, np.mean(M1_HR_stds_af))
print()
print('Heart rate mean AFTER ',EX,' exam \n', M1_HR_mean_af, np.mean(M1_HR_mean_af))
print('Heart rate std AFTER ',EX,' exam \n', M1_HR_std_af, np.mean(M1_HR_std_af))
print()
print('Heart rate long mean AFTER ',EX,' exam \n', M1_HR_long_af, np.mean(M1_HR_long_af))
print('Heart rate long std AFTER ',EX,' exam \n', M1_HR_stdl_af, np.mean(M1_HR_stdl_af))

After this is optional. Plotting the exam length signals

In [ ]:
for stud in range(10):
    data = np.loadtxt('exam_S'+str(stud)+'_M1.csv', delimiter=',', skiprows=1)
    time = data[:,0]
    BVP = data[:,1]
    HR = data[:,3]

    fig, ax = plt.subplots(figsize=(10,10))

    ax.plot(time, BVP, label='BVP')
    ax.plot(time, HR, label='HR')
    plt.title('Student '+str(stud+1)+' BVP and HR')
    plt.legend()

In [ ]:
for stud in range(10):
    data = np.loadtxt('exam_S'+str(stud)+'_M2.csv', delimiter=',', skiprows=1)
    time = data[:,0]
    BVP = data[:,1]
    HR = data[:,3]

    fig, ax = plt.subplots(figsize=(10,10))

    ax.plot(time, BVP, label='BVP')
    ax.plot(time, HR, label='HR')
    plt.title('Student '+str(stud+1)+' BVP and HR')
    plt.legend()

In [ ]:
for stud in range(10):
    data = np.loadtxt('exam_S'+str(stud)+'_F.csv', delimiter=',', skiprows=1)
    time = data[:,0]
    BVP = data[:,1]
    HR = data[:,3]

    fig, ax = plt.subplots(figsize=(10,10))

    ax.plot(time, BVP, label='BVP')
    ax.plot(time, HR, label='HR')
    plt.title('Student '+str(stud+1)+' BVP and HR')
    plt.legend()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

for stud in range(10):
    data = np.loadtxt('exam_S'+str(stud)+'_M1.csv', delimiter=',', skiprows=1) #+str(stud)+
    time = data[:,0]
    EDA = data[:,2] 
    IBI = data[:,5]

    fig, ax = plt.subplots(figsize=(10,10))

    ax.plot(time, EDA, label='EDA')
    ax.plot(time, IBI, label='IBI')
    plt.title('Student '+str(stud+1)+' EDA and IBI') 
    plt.legend()

In [ ]:
for stud in range(10):
    data = np.loadtxt('exam_S'+str(stud)+'_M2.csv', delimiter=',', skiprows=1)
    time = data[:,0]
    EDA = data[:,2] 
    IBI = data[:,5]

    fig, ax = plt.subplots(figsize=(10,10))

    ax.plot(time, EDA, label='EDA')
    ax.plot(time, IBI, label='IBI')
    plt.title('Student '+str(stud+1)+' EDA and IBI')
    plt.legend()

In [ ]:
for stud in range(10):
    data = np.loadtxt('exam_S'+str(stud)+'_F.csv', delimiter=',', skiprows=1)
    time = data[:,0]
    EDA = data[:,2] 
    IBI = data[:,5]

    fig, ax = plt.subplots(figsize=(10,10))

    ax.plot(time, EDA, label='EDA')
    ax.plot(time, IBI, label='IBI')
    plt.title('Student '+str(stud+1)+' EDA and IBI')
    plt.legend()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

for stud in range(10):
    data = np.loadtxt('exam_S'+str(stud)+'_M1.csv', delimiter=',', skiprows=1)
    time = data[:,0]
    TEMP = data[:,4] 

    fig, ax = plt.subplots(figsize=(10,10))

    ax.plot(time, TEMP, label='TEMP')
    plt.title('Student '+str(stud+1)+' TEMP')
    plt.legend()

In [ ]:
for stud in range(10):
    data = np.loadtxt('exam_S'+str(stud)+'_M2.csv', delimiter=',', skiprows=1)
    time = data[:,0]
    TEMP = data[:,4] 

    fig, ax = plt.subplots(figsize=(10,10))

    ax.plot(time, TEMP, label='TEMP')
    plt.title('Student '+str(stud+1)+' TEMP')
    plt.legend()

In [ ]:
for stud in range(10):
    data = np.loadtxt('exam_S'+str(stud)+'_F.csv', delimiter=',', skiprows=1)
    time = data[:,0]
    TEMP = data[:,4] 

    fig, ax = plt.subplots(figsize=(10,10))

    ax.plot(time, TEMP, label='TEMP')
    plt.title('Student '+str(stud+1)+' TEMP')
    plt.legend()